##### Grading Feedback

# IST 718: Big Data Analytics

- Professor: Willard Williamson <wewillia@syr.edu>
- Faculty Assistant: Vidushi Mishra <vmishr01@syr.edu>
- Faculty Assistant: Pranav Kottoli Radhakrishna <pkottoli@syr.edu>
## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers from your classmates.  Short code snippets are allowed from the internet.  Code from the class text books or class provided code can be copied in its entirety.__
- Do not modify cells marked as grading cells or marked as do not modify.
- Before submitting your work, remember to check for run time errors with the following procedure:
`Runtime `$\rightarrow$ Factory reset runtime followed by Runtime $\rightarrow$ Run All.  All runtime errors will result in a minimum penalty of half off.
- Google Colab is the official class runtime environment so you should test your code on Colab before submission.
- All plots shall include descriptive title and axis labels.  Plot legends shall be included where possible.  Unless stated otherwise, plots can be made using any Python plotting package.  It is understood that spark data structures must be converted to something like numpy or pandas prior to making plots.  All required mathematical operations, filtering, selection, etc., required by a homework question shall be performed in spark prior to converting to numpy or pandas.
- Grading feedback cells are there for graders to provide feedback to students.  Don't change or remove grading feedback cells.
- Don't add or remove files from your git repo.
- Do not change file names in your repo.  This also means don't change the title of the ipython notebook.
- You are free to add additional code cells around the cells marked `your code here`.
- We reserve the right to take points off for operations that are extremely inefficient or "heavy weight".  This is a big data class and extremely inefficient operations make a big difference when scaling up to large data sets.  For example, the spark dataframe collect() method is a very heavy weight operation and should not be used unless it there is a real need for it.  An example where collect() might be needed is to get ready to make a plot after filtering a spark dataframe.
- import * is not allowed because it is considered a very bad coding practice and in some cases can result in a significant delay (which slows down the grading process) in loading imports.  For example, the statement `from sympy import *` is not allowed.  You must import the specific packages that you need. 
- The graders reserve the right to deduct points for subjective things we see with your code.  For example, if we ask you to create a pandas data frame to display values from an investigation and you hard code the values, we will take points off for that.  This is only one of many different things we could find in reviewing your code.  In general, write your code like you are submitting it for a code peer review in industry.  
- Level of effort is part of our subjective grading.  For example, in cases where we ask for a more open ended investigation, some students put in significant effort and some students do the minimum possible to meet requirements.  In these cases, we may take points off for students who did not put in much effort as compared to students who put in a lot of effort.  We feel that the students who did a better job deserve a better grade.  We reserve the right to invoke level of effort grading at any time.
- Only use spark, spark machine learning, spark data frames, RDD's, and map reduce to solve all problems unless instructed otherwise.
- Your notebook must run from start to finish without requiring manual input by the graders.  For example, do not mount your personal Google drive in your notebook as this will require graders to perform manual steps.  In short, your notebook should run from start to finish with no runtime errors and no need for graders to perform any manual steps.


## Note that this notebook is expected to run in the Google Colab environment.  All grading for this assignment will take place exclusively in Google Colab.

This homework proves that diamonds are forever.  In homework 3, we used linear regression to predict diamond prices and evaluated model performance using MSE as the scoring metric.  In this homework, we are going to use the same diamonds data set but this time use decision trees and deep learning to see if we can improve upon the linear regression performance from homework 3.

# Diamonds Data
Just to prove that diamonds are forever, we are going to revisit the diamonds data set.  This homework assignment will use the diamonds dataset to explore random forest decision tree models.

The diamonds.csv data set contains 10 columns:
- carat: Carat weight of the diamond
- cut: Describes cut quality of the diamond. Quality in increasing order Fair, Good, Very Good, Premium, Ideal
- color: Color of the diamond, with D being the best and J the worst
- clarity: How obvious inclusions are within the diamond:(in order from best to worst, FL = flawless, I3= level 3 inclusions) FL,IF, VVS1, etc.  See this web site for an exhaustive ranking of [clarity](https://4cs.gia.edu/en-us/diamond-clarity/?gclid=Cj0KCQjwnqH7BRDdARIsACTSAduMoc2KQbXkO94BxCfBNC5X8YyjAYcFpWThKQMW46cQj_3p0pZ0o84aAuagEALw_wcB).  The web site has a nice sliding scale you can drag to see the relationship between clarity grades.
- depth: depth % - The height of a diamond, measured from the culet to the table, divided by its average girdle diameter
- table: table% -  The width of the diamond's table expressed as a percentage of its average diameter
- price: The price of the diamond
- x: Length (mm)
- y: Width (mm)
- z: Height (mm)

In [12]:
# Grading Cell
enable_grid_search = False

The following cell is used to read the diamonds data set into the colab environment.  Do not change or modify the following cell.

In [2]:
%%bash
# Do not change or modify this file
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already isntalled
pip install pyspark

# Download the data files from github
# If the data file does not exist in the colab environment
if [[ ! -f ./quotes_by_char.csv ]]; then 
   # download the data file from github and save it in this colab environment instance
   wget https://raw.githubusercontent.com/wewilli1/ist718_data/master/diamonds.csv  
fi

--2020-11-26 17:20:57--  https://raw.githubusercontent.com/wewilli1/ist718_data/master/diamonds.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3192560 (3.0M) [text/plain]
Saving to: ‘diamonds.csv.4’

     0K .......... .......... .......... .......... ..........  1% 3.80M 1s
    50K .......... .......... .......... .......... ..........  3% 10.8M 1s
   100K .......... .......... .......... .......... ..........  4% 4.10M 1s
   150K .......... .......... .......... .......... ..........  6% 39.8M 0s
   200K .......... .......... .......... .......... ..........  8% 5.77M 0s
   250K .......... .......... .......... .......... ..........  9% 31.2M 0s
   300K .......... .......... .......... .......... .......... 11% 24.4M 0s
   350K .......... .......... ......

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
#spark = SparkSession.builder.getOrCreate()
spark = SparkSession \
  .builder \
  .master("local[*]")\
  .config("spark.executor.memory", "70g") \
  .config("spark.driver.memory", "50g")\
  .config("spark.memory.offHeap.enabled",'true')\
  .config("spark.memory.offHeap.size","16g")\
  .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Question 0 (0 pts)
Please provide the following the data so we can easily correlate your notebook with the grade book:
- Your Name: Rahul Wable
- Your github user name: rdwable
- Your SU email address: rdwable@syr.edu

Your grade for grid search problems in this assignment will be determined in part on level of effort and your model performance results as compared to other students in the class.

# Question 1 (10 pts)
Read the diamonds.csv file into a spark data frame named `diamonds_df`.  Perform feature engineering as needed for training decision trees.  Name the new data frame diamonds_df_xformed.

In [4]:
# your code here
diamonds_df= spark.read.format("csv").option("header", "true").load("diamonds.csv")
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
diamonds_df = diamonds_df.drop('_c0')
#Converting columns from string to float
diamonds_df=diamonds_df.withColumn('carat',diamonds_df['carat'].cast("float").alias('carat'))
diamonds_df=diamonds_df.withColumn('depth',diamonds_df['depth'].cast("float").alias('depth'))
diamonds_df=diamonds_df.withColumn('table',diamonds_df['table'].cast("float").alias('table'))
diamonds_df=diamonds_df.withColumn('price',diamonds_df['price'].cast("float").alias('price'))
diamonds_df=diamonds_df.withColumn('x',diamonds_df['x'].cast("float").alias('x'))
diamonds_df=diamonds_df.withColumn('y',diamonds_df['y'].cast("float").alias('y'))
diamonds_df=diamonds_df.withColumn('z',diamonds_df['z'].cast("float").alias('z'))
diamonds_df_xformed = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(diamonds_df) for column in list(set(diamonds_df.columns)-set(['carat'])-set(['depth'])-set(['table'])-set(['price'])-set(['x'])-set(['y'])-set(['z']))]
feature_engineering_pipe = Pipeline(stages=diamonds_df_xformed)
diamonds_pipe = feature_engineering_pipe.fit(diamonds_df)
diamonds_df_xformed= diamonds_pipe.transform(diamonds_df)
#Dropping the earlier columns
diamonds_df_xformed= diamonds_df_xformed.drop('cut')
diamonds_df_xformed = diamonds_df_xformed.drop('color')
diamonds_df_xformed = diamonds_df_xformed.drop('clarity')

#Renaming the columns
diamonds_df_xformed = diamonds_df_xformed.withColumnRenamed("cut_index","cut")
diamonds_df_xformed = diamonds_df_xformed.withColumnRenamed("color_index","color")
diamonds_df_xformed = diamonds_df_xformed.withColumnRenamed("clarity_index","clarity")


diamonds_df_xformed.show()



+-----+-----+-----+-----+----+----+----+---+-------+-----+
|carat|depth|table|price|   x|   y|   z|cut|clarity|color|
+-----+-----+-----+-----+----+----+----+---+-------+-----+
| 0.23| 61.5| 55.0|326.0|3.95|3.98|2.43|0.0|    2.0|  1.0|
| 0.21| 59.8| 61.0|326.0|3.89|3.84|2.31|1.0|    0.0|  1.0|
| 0.23| 56.9| 65.0|327.0|4.05|4.07|2.31|3.0|    3.0|  1.0|
| 0.29| 62.4| 58.0|334.0| 4.2|4.23|2.63|1.0|    1.0|  5.0|
| 0.31| 63.3| 58.0|335.0|4.34|4.35|2.75|3.0|    2.0|  6.0|
| 0.24| 62.8| 57.0|336.0|3.94|3.96|2.48|2.0|    4.0|  6.0|
| 0.24| 62.3| 57.0|336.0|3.95|3.98|2.47|2.0|    5.0|  5.0|
| 0.26| 61.9| 55.0|337.0|4.07|4.11|2.53|2.0|    0.0|  3.0|
| 0.22| 65.1| 61.0|337.0|3.87|3.78|2.49|4.0|    1.0|  1.0|
| 0.23| 59.4| 61.0|338.0| 4.0|4.05|2.39|2.0|    3.0|  3.0|
|  0.3| 64.0| 55.0|339.0|4.25|4.28|2.73|3.0|    0.0|  6.0|
| 0.23| 62.8| 56.0|340.0|3.93| 3.9|2.46|0.0|    3.0|  6.0|
| 0.22| 60.4| 61.0|342.0|3.88|3.84|2.33|1.0|    0.0|  2.0|
| 0.31| 62.2| 54.0|344.0|4.35|4.37|2.71|0.0|    2.0|  6.

In [5]:
# Grading Cell - do not modify
display(diamonds_df_xformed.toPandas().head())

,carat,depth,table,price,x,y,z,cut,clarity,color
0,0.23,61.500000,55.0,326.0,3.95,3.98,2.43,0.0,2.0,1.0
1,0.21,59.799999,61.0,326.0,3.89,3.84,2.31,1.0,0.0,1.0
2,0.23,56.900002,65.0,327.0,4.05,4.07,2.31,3.0,3.0,1.0
3,0.29,62.400002,58.0,334.0,4.20,4.23,2.63,1.0,1.0,5.0
4,0.31,63.299999,58.0,335.0,4.34,4.35,2.75,3.0,2.0,6.0


##### Grading Feedback Cell

The following questions will create a random forest regressor model, train the model using a grid search, and use the model for inference.  The goal is to see if we can improve upon the linear regression score from homework 3. You can find the spark documentation for the random forest regressor [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-regression).

# Question 2 (20 pts)
Create and train your random forest regressor model using a grid search in the cell below.  You are free to use K-Fold Cross validation if you wish.  Your grid search must be entirely encapsulated in the `if enable_grid_search` if statement.  The `enable_grid_search` Boolean is defined in a grading cell above.  You will disable the grid search before you submit by setting enable_grid_search to false.  Setting enable_grid_search to false should not result in a runtime error.  You will not receive full credit if any part of your grid search is outside of the if statement or if runtime errros result from setting the `enable_grid_search` variable to false.

In [6]:
# your code here

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import feature
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import BinaryClassificationEvaluator
training_df, testing_df = diamonds_df_xformed.randomSplit([0.7, 0.3],seed=30)
vc = feature.VectorAssembler(inputCols=['carat','depth','table','price','x','y','z','clarity','color','cut'], outputCol='features')
rf = RandomForestRegressor(featuresCol='features',labelCol='price')
rf_pipeline = Pipeline(stages=[vc, rf])
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")

if enable_grid_search:
  paramGrid = (ParamGridBuilder().addGrid(rf.numTrees,[5,20,40]).addGrid(rf.maxDepth,[2,5,10]).addGrid(rf.bootstrap,[True,False])).build()
  crossval= CrossValidator(estimator= rf_pipeline,estimatorParamMaps=paramGrid,evaluator=evaluator,numFolds=3)
  output = crossval.fit(training_df)
  #print('MSE:', evaluator.evaluate(output))
  print('mse',output.avgMetrics)
  best_index= np.argmin(output.avgMetrics)
  print(paramGrid[best_index])

pass

mse [1338442.3465281508, 1379623.0393133701, 299716.39819425566, 305560.7524483915, 84619.17990312002, 80122.23124065192, 1041489.4394886892, 1061832.7046167674, 186417.3376663087, 186648.36924362154, 70257.15286506625, 67957.25693614788, 1139512.5428198753, 1161929.4806193411, 173611.82755714087, 173954.75136644035, 67714.70257097752, 66136.21724598126]
{Param(parent='RandomForestRegressor_f0a4bcc7d194', name='numTrees', doc='Number of trees to train (>= 1).'): 40, Param(parent='RandomForestRegressor_f0a4bcc7d194', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10, Param(parent='RandomForestRegressor_f0a4bcc7d194', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): False}


##### Grading Feedback Cell

# Question 3 (20 pts)
Create a pipeline named `best_pipe` that hard codes the tuning parameters from the best model found by the grid search in question 2 above.  Train and test best_pipe.  Do not use k-fold cross validation in question 3.  Clearly print the resulting train and test MSE for best_pipe so it's easy for the graders to see your resulting MSEs.

In [7]:
# Your code here
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.mllib.tree import RandomForest, RandomForestModel


num_trees = 40 
maxdepth= 10
train, test = diamonds_df_xformed.randomSplit([0.7, 0.3])
best_rf = RandomForestRegressor(featuresCol='features',labelCol='price',maxDepth=maxdepth,numTrees =num_trees)
best_pipe = Pipeline(stages =[vc,best_rf])
best_model = best_pipe.fit(training_df)
predictions = best_model.transform(testing_df)

evaluator = RegressionEvaluator(labelCol='price', predictionCol="prediction", metricName="mse")
print(f'Train MSE for best model is {evaluator.evaluate(best_model.transform(train))}')
print(f'Test MSE for bes model is {evaluator.evaluate(predictions)}')



Train MSE for best model is 59599.2120630911
Test MSE for bes model is 73448.77466708544


##### Grading Feedback Cell

# Question 4 (20 pts)
Use your best_pipe pipeline in question 3 for inference.  Create a pandas data frame named `rf_feature_importance` which contains 2 columns: `feature`, and `importance`.  Load the feature column with the feature name and the importance column with the feature importance score as determined by the random forest model. Sort the feature importances from high to low such that the most important feature is in the first row of the data frame.

In [8]:
# your code here
import pandas as pd
from pyspark.sql.functions import concat, col, lit
inference_pipe = Pipeline(stages=[vc,rf])   #Pipelining the data
c = inference_pipe.fit(train)
bestmodel= best_model.stages[-1]
value= bestmodel.featureImportances.toArray()  #Extracting coefficient values
coefficient=c.stages[0].getInputCols()         #Extracting coefiicient names
rf_feature_importance=pd.DataFrame(coefficient,columns=['feature'])    #Adding coefficient names in a pandas data frame
rf_feature_importance.insert(1,"importance",value,True)  #Inserting coefficient values in the data frame
display(rf_feature_importance)

,feature,importance
0,carat,0.233516
1,depth,0.000480
2,table,0.000247
3,price,0.411568
4,x,0.078266
5,y,0.219672
6,z,0.043868
7,clarity,0.007992
8,color,0.003738
9,cut,0.000652


In [9]:
# grading cell - do not modify
display(rf_feature_importance)

,feature,importance
0,carat,0.233516
1,depth,0.000480
2,table,0.000247
3,price,0.411568
4,x,0.078266
5,y,0.219672
6,z,0.043868
7,clarity,0.007992
8,color,0.003738
9,cut,0.000652


##### Grading Feedback Cell

# Question 5 (20 pts)
Write code to print the decision logic for any of the trees in the forest from the best_pipe pipeline.  Copy the printed decision text to the tree printout markdown cell below and retain the same formatting and indentation as the code printout so it's easy for the graders to view the data.  You need to double click the "Your Decision Tree Print Out Here" markdown cell and paste your output inside the two sets of triple quotes. The triple quotes are jupyter markdown indicating you want to present code.  Essentially, replace the text inside the triple quotes with your tree printout.  Solutions that do not maintain readable formatting will not receive full credit.

Add comments to the markdown cell below describing how the root node is split:  Describe 2 things in the markdown cell.  1) What specific predictor variable is being split and what is the value that determines the left / right split.  2) We need you to paste the tree decision logic output from your run in the markdown cell because the top level split may change from run to run.  If the graders run your notebook, the top level split for the tree may be different than the top level split from when you made the run.  Describe why the top level predictor changes from run to run.


In [11]:
# your code here
print(bestmodel.trees[0].toDebugString)


DecisionTreeRegressionModel: uid=dtr_62fe96359c1f, depth=10, numNodes=1807, numFeatures=10
  If (feature 0 <= 0.9950000047683716)
   If (feature 4 <= 5.605000019073486)
    If (feature 3 <= 1305.5)
     If (feature 0 <= 0.375)
      If (feature 3 <= 753.5)
       If (feature 7 in {0.0,2.0,7.0})
        If (feature 3 <= 572.5)
         If (feature 8 in {0.0,2.0,3.0,5.0,6.0})
          If (feature 9 in {0.0,2.0,3.0})
           If (feature 3 <= 491.5)
            Predict: 431.328320802005
           Else (feature 3 > 491.5)
            Predict: 528.6122448979592
          Else (feature 9 not in {0.0,2.0,3.0})
           If (feature 3 <= 491.5)
            Predict: 445.11764705882354
           Else (feature 3 > 491.5)
            Predict: 534.8181818181819
         Else (feature 8 not in {0.0,2.0,3.0,5.0,6.0})
          If (feature 7 in {2.0,7.0})
           If (feature 3 <= 491.5)
            Predict: 449.21505376344084
           Else (feature 3 > 491.5)
            Predict: 516.083333

```
Your Decision Tree Print Out Here - Replace this text with the tree decision logic:
DecisionTreeRegressionModel: uid=dtr_62fe96359c1f, depth=10, numNodes=1807, numFeatures=10
  If (feature 0 <= 0.9950000047683716)
   If (feature 4 <= 5.605000019073486)
    If (feature 3 <= 1305.5)
     If (feature 0 <= 0.375)
      If (feature 3 <= 753.5)
       If (feature 7 in {0.0,2.0,7.0})
        If (feature 3 <= 572.5)
         If (feature 8 in {0.0,2.0,3.0,5.0,6.0})
          If (feature 9 in {0.0,2.0,3.0})
           If (feature 3 <= 491.5)
            Predict: 431.328320802005
           Else (feature 3 > 491.5)
            Predict: 528.6122448979592
          Else (feature 9 not in {0.0,2.0,3.0})
           If (feature 3 <= 491.5)
            Predict: 445.11764705882354
           Else (feature 3 > 491.5)
            Predict: 534.8181818181819
         Else (feature 8 not in {0.0,2.0,3.0,5.0,6.0})
          If (feature 7 in {2.0,7.0})
           If (feature 3 <= 491.5)
            Predict: 449.21505376344084
           Else (feature 3 > 491.5)
            Predict: 516.0833333333334
          Else (feature 7 not in {2.0,7.0})
           If (feature 8 in {1.0})
            Predict: 529.0481927710844
           Else (feature 8 not in {1.0})
            Predict: 552.2954545454545
        Else (feature 3 > 572.5)
         If (feature 3 <= 630.5)
          If (feature 4 <= 4.444999933242798)
           If (feature 2 <= 53.95000076293945)
            Predict: 630.0
           Else (feature 2 > 53.95000076293945)
            Predict: 589.3006993006993
          Else (feature 4 > 4.444999933242798)
           If (feature 2 <= 55.04999923706055)
            Predict: 592.9459459459459
           Else (feature 2 > 55.04999923706055)
            Predict: 603.7391304347826
         Else (feature 3 > 630.5)
          If (feature 3 <= 689.5)
           If (feature 7 in {2.0})
            Predict: 649.2727272727273
           Else (feature 7 not in {2.0})
            Predict: 660.3333333333334
          Else (feature 3 > 689.5)
           If (feature 0 <= 0.33500000834465027)
            Predict: 719.5847457627119
           Else (feature 0 > 0.33500000834465027)
            Predict: 737.6521739130435
       Else (feature 7 not in {0.0,2.0,7.0})
        If (feature 0 <= 0.29500000178813934)
         If (feature 3 <= 572.5)
          If (feature 3 <= 491.5)
           If (feature 7 in {1.0,3.0,6.0})
            Predict: 430.23643410852713
           Else (feature 7 not in {1.0,3.0,6.0})
            Predict: 459.85714285714283
          Else (feature 3 > 491.5)
           If (feature 8 in {3.0,5.0})
            Predict: 517.9591836734694
           Else (feature 8 not in {3.0,5.0})
            Predict: 532.2629482071713
         Else (feature 3 > 572.5)
          If (feature 5 <= 4.255000114440918)
           If (feature 2 <= 57.04999923706055)
            Predict: 625.9444444444445
           Else (feature 2 > 57.04999923706055)
            Predict: 611.1683168316831
          Else (feature 5 > 4.255000114440918)
           If (feature 3 <= 630.5)
            Predict: 607.0
           Else (feature 3 > 630.5)
            Predict: 675.2333333333333
        Else (feature 0 > 0.29500000178813934)
         If (feature 7 in {1.0,3.0})
          If (feature 8 in {3.0,5.0,6.0})
           If (feature 3 <= 572.5)
            Predict: 496.1021377672209
           Else (feature 3 > 572.5)
            Predict: 654.4763513513514
          Else (feature 8 not in {3.0,5.0,6.0})
           If (feature 3 <= 630.5)
            Predict: 591.0340909090909
           Else (feature 3 > 630.5)
            Predict: 695.7911001236093
         Else (feature 7 not in {1.0,3.0})
          If (feature 0 <= 0.32500000298023224)
           If (feature 9 in {0.0})
            Predict: 642.9835390946502
           Else (feature 9 not in {0.0})
            Predict: 667.8643410852713
          Else (feature 0 > 0.32500000298023224)
           If (feature 6 <= 2.84499990940094)
            Predict: 684.5964912280701
           Else (feature 6 > 2.84499990940094)
            Predict: 597.25
      Else (feature 3 > 753.5)
       If (feature 6 <= 2.7450000047683716)
        If (feature 7 in {0.0,1.0,3.0})
         If (feature 0 <= 0.3050000071525574)
          If (feature 1 <= 62.95000076293945)
           If (feature 3 <= 813.5)
            Predict: 777.4925373134329
           Else (feature 3 > 813.5)
            Predict: 863.1526717557252
          Else (feature 1 > 62.95000076293945)
           If (feature 9 in {2.0,3.0})
            Predict: 776.0
           Else (feature 9 not in {2.0,3.0})
            Predict: 799.0
         Else (feature 0 > 0.3050000071525574)
          If (feature 8 in {0.0,2.0,3.0})
           If (feature 0 <= 0.3149999976158142)
            Predict: 803.054347826087
           Else (feature 0 > 0.3149999976158142)
            Predict: 833.8842975206611
          Else (feature 8 not in {0.0,2.0,3.0})
           If (feature 3 <= 882.5)
            Predict: 805.1785714285714
           Else (feature 3 > 882.5)
            Predict: 951.022346368715
        Else (feature 7 not in {0.0,1.0,3.0})
         If (feature 9 in {0.0,2.0,3.0})
          If (feature 3 <= 956.5)
           If (feature 4 <= 4.305000066757202)
            Predict: 818.6682692307693
           Else (feature 4 > 4.305000066757202)
            Predict: 858.6466431095406
          Else (feature 3 > 956.5)
           If (feature 5 <= 4.365000009536743)
            Predict: 1026.7739130434782
           Else (feature 5 > 4.365000009536743)
            Predict: 1053.68345323741
         Else (feature 9 not in {0.0,2.0,3.0})
          If (feature 3 <= 956.5)
           If (feature 3 <= 882.5)
            Predict: 812.4
           Else (feature 3 > 882.5)
            Predict: 928.4035087719299
          Else (feature 3 > 956.5)
           If (feature 3 <= 1045.0)
            Predict: 999.9255319148937
           Else (feature 3 > 1045.0)
            Predict: 1101.5892857142858
       Else (feature 6 > 2.7450000047683716)
        If (feature 3 <= 956.5)
         If (feature 3 <= 813.5)
          If (feature 5 <= 4.515000104904175)
           If (feature 2 <= 55.04999923706055)
            Predict: 772.3125
           Else (feature 2 > 55.04999923706055)
            Predict: 782.7710843373494
          Else (feature 5 > 4.515000104904175)
           If (feature 8 in {0.0,5.0})
            Predict: 778.6
           Else (feature 8 not in {0.0,5.0})
            Predict: 788.6637168141593
         Else (feature 3 > 813.5)
          If (feature 3 <= 882.5)
           If (feature 8 in {0.0,1.0,3.0,4.0})
            Predict: 846.0
           Else (feature 8 not in {0.0,1.0,3.0,4.0})
            Predict: 861.8392857142857
          Else (feature 3 > 882.5)
           If (feature 1 <= 60.85000038146973)
            Predict: 908.4705882352941
           Else (feature 1 > 60.85000038146973)
            Predict: 922.5314685314685
        Else (feature 3 > 956.5)
         If (feature 3 <= 1045.0)
          If (feature 6 <= 2.784999966621399)
           If (feature 0 <= 0.33500000834465027)
            Predict: 986.9555555555555
           Else (feature 0 > 0.33500000834465027)
            Predict: 1002.8571428571429
          Else (feature 6 > 2.784999966621399)
           If (feature 1 <= 61.95000076293945)
            Predict: 999.5652173913044
           Else (feature 1 > 61.95000076293945)
            Predict: 1015.3061224489796
         Else (feature 3 > 1045.0)
          If (feature 3 <= 1145.5)
           If (feature 0 <= 0.3149999976158142)
            Predict: 1046.0
           Else (feature 0 > 0.3149999976158142)
            Predict: 1091.4313725490197
          Else (feature 3 > 1145.5)
           If (feature 4 <= 4.515000104904175)
            Predict: 1231.2
           Else (feature 4 > 4.515000104904175)
            Predict: 1203.75
     Else (feature 0 > 0.375)
      If (feature 4 <= 4.894999980926514)
       If (feature 7 in {0.0,1.0,2.0,7.0})
        If (feature 3 <= 813.5)
         If (feature 3 <= 689.5)
          If (feature 3 <= 630.5)
           If (feature 3 <= 572.5)
            Predict: 532.6969696969697
           Else (feature 3 > 572.5)
            Predict: 604.816091954023
          Else (feature 3 > 630.5)
           If (feature 0 <= 0.39499999582767487)
            Predict: 656.1698113207547
           Else (feature 0 > 0.39499999582767487)
            Predict: 670.5343511450382
         Else (feature 3 > 689.5)
          If (feature 7 in {0.0})
           If (feature 6 <= 2.965000033378601)
            Predict: 729.1019108280254
           Else (feature 6 > 2.965000033378601)
            Predict: 749.6267605633802
          Else (feature 7 not in {0.0})
           If (feature 0 <= 0.41499999165534973)
            Predict: 764.4240506329114
           Else (feature 0 > 0.41499999165534973)
            Predict: 738.3529411764706
        Else (feature 3 > 813.5)
         If (feature 4 <= 4.7149999141693115)
          If (feature 3 <= 956.5)
           If (feature 1 <= 62.45000076293945)
            Predict: 878.8987341772151
           Else (feature 1 > 62.45000076293945)
            Predict: 905.082191780822
          Else (feature 3 > 956.5)
           If (feature 3 <= 1045.0)
            Predict: 994.8333333333334
           Else (feature 3 > 1045.0)
            Predict: 1102.3333333333333
         Else (feature 4 > 4.7149999141693115)
          If (feature 1 <= 62.95000076293945)
           If (feature 3 <= 956.5)
            Predict: 883.7120743034055
           Else (feature 3 > 956.5)
            Predict: 1041.0323529411764
          Else (feature 1 > 62.95000076293945)
           If (feature 0 <= 0.4350000023841858)
            Predict: 901.2173913043479
           Else (feature 0 > 0.4350000023841858)
            Predict: 1023.6111111111111
       Else (feature 7 not in {0.0,1.0,2.0,7.0})
        If (feature 4 <= 4.7149999141693115)
         If (feature 2 <= 55.95000076293945)
          If (feature 3 <= 956.5)
           If (feature 3 <= 813.5)
            Predict: 721.475
           Else (feature 3 > 813.5)
            Predict: 890.7317073170732
          Else (feature 3 > 956.5)
           If (feature 3 <= 1145.5)
            Predict: 1063.566037735849
           Else (feature 3 > 1145.5)
            Predict: 1222.4285714285713
         Else (feature 2 > 55.95000076293945)
          If (feature 3 <= 956.5)
           If (feature 8 in {6.0})
            Predict: 708.625
           Else (feature 8 not in {6.0})
            Predict: 848.2696629213483
          Else (feature 3 > 956.5)
           If (feature 1 <= 59.95000076293945)
            Predict: 1006.3333333333334
           Else (feature 1 > 59.95000076293945)
            Predict: 1103.1060606060605
        Else (feature 4 > 4.7149999141693115)
         If (feature 3 <= 1045.0)
          If (feature 3 <= 882.5)
           If (feature 9 in {0.0,3.0})
            Predict: 760.2696629213483
           Else (feature 9 not in {0.0,3.0})
            Predict: 816.3157894736842
          Else (feature 3 > 882.5)
           If (feature 3 <= 956.5)
            Predict: 919.8595041322315
           Else (feature 3 > 956.5)
            Predict: 1000.7860465116279
         Else (feature 3 > 1045.0)
          If (feature 0 <= 0.41499999165534973)
           If (feature 1 <= 62.95000076293945)
            Predict: 1142.433155080214
           Else (feature 1 > 62.95000076293945)
            Predict: 1085.6
          Else (feature 0 > 0.41499999165534973)
           If (feature 3 <= 1145.5)
            Predict: 1102.6136363636363
           Else (feature 3 > 1145.5)
            Predict: 1222.132450331126
      Else (feature 4 > 4.894999980926514)
       If (feature 5 <= 5.105000019073486)
        If (feature 3 <= 1045.0)
         If (feature 2 <= 55.04999923706055)
          If (feature 1 <= 66.54999923706055)
           If (feature 7 in {2.0})
            Predict: 817.8888888888889
           Else (feature 7 not in {2.0})
            Predict: 913.953488372093
          Else (feature 1 > 66.54999923706055)
           Predict: 613.0
         Else (feature 2 > 55.04999923706055)
          If (feature 7 in {7.0})
           If (feature 3 <= 813.5)
            Predict: 688.8333333333334
           Else (feature 3 > 813.5)
            Predict: 930.0
          Else (feature 7 not in {7.0})
           If (feature 6 <= 2.915000081062317)
            Predict: 805.0833333333334
           Else (feature 6 > 2.915000081062317)
            Predict: 957.7378048780488
        Else (feature 3 > 1045.0)
         If (feature 3 <= 1145.5)
          If (feature 9 in {2.0})
           If (feature 1 <= 63.25)
            Predict: 1080.6842105263158
           Else (feature 1 > 63.25)
            Predict: 1060.2
          Else (feature 9 not in {2.0})
           If (feature 6 <= 3.3049999475479126)
            Predict: 1092.9878048780488
           Else (feature 6 > 3.3049999475479126)
            Predict: 1052.0
         Else (feature 3 > 1145.5)
          If (feature 7 in {0.0,4.0,6.0})
           If (feature 8 in {0.0,1.0,3.0})
            Predict: 1196.7246376811595
           Else (feature 8 not in {0.0,1.0,3.0})
            Predict: 1224.9354838709678
          Else (feature 7 not in {0.0,4.0,6.0})
           If (feature 9 in {1.0,2.0,3.0,4.0})
            Predict: 1217.4155844155844
           Else (feature 9 not in {1.0,2.0,3.0,4.0})
            Predict: 1257.84375
       Else (feature 5 > 5.105000019073486)
        If (feature 7 in {7.0})
         If (feature 4 <= 5.365000009536743)
          If (feature 8 in {2.0,3.0})
           If (feature 1 <= 62.14999961853027)
            Predict: 829.6666666666666
           Else (feature 1 > 62.14999961853027)
            Predict: 870.0
          Else (feature 8 not in {2.0,3.0})
           Predict: 938.0
         Else (feature 4 > 5.365000009536743)
          If (feature 3 <= 1045.0)
           If (feature 9 in {1.0})
            Predict: 873.0
           Else (feature 9 not in {1.0})
            Predict: 980.5882352941177
          Else (feature 3 > 1045.0)
           If (feature 0 <= 0.6949999928474426)
            Predict: 1255.0
           Else (feature 0 > 0.6949999928474426)
            Predict: 1100.125
        Else (feature 7 not in {7.0})
         If (feature 3 <= 1145.5)
          If (feature 9 in {4.0})
           If (feature 5 <= 5.184999942779541)
            Predict: 851.0
           Else (feature 5 > 5.184999942779541)
            Predict: 1041.0
          Else (feature 9 not in {4.0})
           If (feature 3 <= 1045.0)
            Predict: 977.4235294117647
           Else (feature 3 > 1045.0)
            Predict: 1100.9709302325582
         Else (feature 3 > 1145.5)
          If (feature 6 <= 3.3049999475479126)
           If (feature 0 <= 0.5049999952316284)
            Predict: 1213.7169811320755
           Else (feature 0 > 0.5049999952316284)
            Predict: 1234.1423841059602
          Else (feature 6 > 3.3049999475479126)
           If (feature 2 <= 56.04999923706055)
            Predict: 1246.860465116279
           Else (feature 2 > 56.04999923706055)
            Predict: 1262.111111111111
    Else (feature 3 > 1305.5)
     If (feature 0 <= 0.574999988079071)
      If (feature 3 <= 1940.5)
       If (feature 3 <= 1584.0)
        If (feature 7 in {0.0,2.0,4.0,5.0,6.0})
         If (feature 8 in {0.0,1.0,2.0,6.0})
          If (feature 1 <= 62.35000038146973)
           If (feature 0 <= 0.5450000166893005)
            Predict: 1392.644578313253
           Else (feature 0 > 0.5450000166893005)
            Predict: 1429.2222222222222
          Else (feature 1 > 62.35000038146973)
           If (feature 8 in {0.0})
            Predict: 1342.2666666666667
           Else (feature 8 not in {0.0})
            Predict: 1379.3978494623657
         Else (feature 8 not in {0.0,1.0,2.0,6.0})
          If (feature 0 <= 0.5049999952316284)
           If (feature 4 <= 4.7850000858306885)
            Predict: 1355.060606060606
           Else (feature 4 > 4.7850000858306885)
            Predict: 1414.7317073170732
          Else (feature 0 > 0.5049999952316284)
           If (feature 5 <= 5.105000019073486)
            Predict: 1414.7222222222222
           Else (feature 5 > 5.105000019073486)
            Predict: 1470.55
        Else (feature 7 not in {0.0,2.0,4.0,5.0,6.0})
         If (feature 6 <= 3.15500009059906)
          If (feature 8 in {3.0,4.0})
           If (feature 4 <= 4.894999980926514)
            Predict: 1342.4
           Else (feature 4 > 4.894999980926514)
            Predict: 1387.1153846153845
          Else (feature 8 not in {3.0,4.0})
           If (feature 1 <= 59.95000076293945)
            Predict: 1398.5217391304348
           Else (feature 1 > 59.95000076293945)
            Predict: 1437.122807017544
         Else (feature 6 > 3.15500009059906)
          If (feature 9 in {3.0,4.0})
           If (feature 8 in {0.0,1.0,3.0,4.0})
            Predict: 1406.6363636363637
           Else (feature 8 not in {0.0,1.0,3.0,4.0})
            Predict: 1473.4666666666667
          Else (feature 9 not in {3.0,4.0})
           If (feature 7 in {1.0})
            Predict: 1471.5639534883721
           Else (feature 7 not in {1.0})
            Predict: 1509.9268292682927
       Else (feature 3 > 1584.0)
        If (feature 7 in {0.0,1.0,2.0,3.0})
         If (feature 8 in {0.0,1.0,2.0,3.0,6.0})
          If (feature 7 in {0.0,1.0,2.0})
           If (feature 3 <= 1754.5)
            Predict: 1661.904
           Else (feature 3 > 1754.5)
            Predict: 1822.478787878788
          Else (feature 7 not in {0.0,1.0,2.0})
           If (feature 3 <= 1754.5)
            Predict: 1669.345238095238
           Else (feature 3 > 1754.5)
            Predict: 1833.2311111111112
         Else (feature 8 not in {0.0,1.0,2.0,3.0,6.0})
          If (feature 3 <= 1754.5)
           If (feature 9 in {0.0,2.0,3.0,4.0})
            Predict: 1663.233870967742
           Else (feature 9 not in {0.0,2.0,3.0,4.0})
            Predict: 1686.28
          Else (feature 3 > 1754.5)
           If (feature 9 in {2.0,3.0})
            Predict: 1812.6857142857143
           Else (feature 9 not in {2.0,3.0})
            Predict: 1845.3657142857144
        Else (feature 7 not in {0.0,1.0,2.0,3.0})
         If (feature 3 <= 1754.5)
          If (feature 1 <= 60.54999923706055)
           If (feature 6 <= 3.15500009059906)
            Predict: 1620.75
           Else (feature 6 > 3.15500009059906)
            Predict: 1741.0
          Else (feature 1 > 60.54999923706055)
           If (feature 7 in {5.0})
            Predict: 1664.6888888888889
           Else (feature 7 not in {5.0})
            Predict: 1696.871794871795
         Else (feature 3 > 1754.5)
          If (feature 9 in {4.0})
           If (feature 1 <= 62.95000076293945)
            Predict: 1791.0
           Else (feature 1 > 62.95000076293945)
            Predict: 1792.0
          Else (feature 9 not in {4.0})
           If (feature 8 in {3.0,4.0,5.0})
            Predict: 1832.6909090909091
           Else (feature 8 not in {3.0,4.0,5.0})
            Predict: 1864.2391304347825
      Else (feature 3 > 1940.5)
       If (feature 3 <= 2422.5)
        If (feature 3 <= 2186.0)
         If (feature 8 in {0.0,1.0,3.0,4.0,5.0})
          If (feature 0 <= 0.5149999856948853)
           If (feature 8 in {0.0,1.0,3.0})
            Predict: 2035.5652173913043
           Else (feature 8 not in {0.0,1.0,3.0})
            Predict: 2069.9444444444443
          Else (feature 0 > 0.5149999856948853)
           If (feature 5 <= 5.265000104904175)
            Predict: 2006.4411764705883
           Else (feature 5 > 5.265000104904175)
            Predict: 2022.5341614906831
         Else (feature 8 not in {0.0,1.0,3.0,4.0,5.0})
          If (feature 7 in {0.0,1.0,3.0,5.0})
           If (feature 1 <= 61.54999923706055)
            Predict: 2103.1
           Else (feature 1 > 61.54999923706055)
            Predict: 2019.2333333333333
          Else (feature 7 not in {0.0,1.0,3.0,5.0})
           If (feature 0 <= 0.5149999856948853)
            Predict: 2056.25
           Else (feature 0 > 0.5149999856948853)
            Predict: 2126.703703703704
        Else (feature 3 > 2186.0)
         If (feature 7 in {0.0,1.0,3.0,5.0})
          If (feature 6 <= 3.15500009059906)
           If (feature 0 <= 0.5049999952316284)
            Predict: 2265.2
           Else (feature 0 > 0.5049999952316284)
            Predict: 2323.5
          Else (feature 6 > 3.15500009059906)
           If (feature 9 in {0.0,2.0})
            Predict: 2255.637931034483
           Else (feature 9 not in {0.0,2.0})
            Predict: 2315.0
         Else (feature 7 not in {0.0,1.0,3.0,5.0})
          If (feature 9 in {0.0,2.0,3.0})
           If (feature 0 <= 0.5450000166893005)
            Predict: 2295.3766233766232
           Else (feature 0 > 0.5450000166893005)
            Predict: 2320.189189189189
          Else (feature 9 not in {0.0,2.0,3.0})
           If (feature 1 <= 59.45000076293945)
            Predict: 2419.0
           Else (feature 1 > 59.45000076293945)
            Predict: 2387.0
       Else (feature 3 > 2422.5)
        If (feature 3 <= 3001.5)
         If (feature 2 <= 55.04999923706055)
          If (feature 3 <= 2689.0)
           If (feature 7 in {4.0})
            Predict: 2506.5
           Else (feature 7 not in {4.0})
            Predict: 2581.714285714286
          Else (feature 3 > 2689.0)
           If (feature 6 <= 3.2350000143051147)
            Predict: 2812.5333333333333
           Else (feature 6 > 3.2350000143051147)
            Predict: 2936.6
         Else (feature 2 > 55.04999923706055)
          If (feature 7 in {1.0,3.0,4.0,6.0})
           If (feature 2 <= 60.14999961853027)
            Predict: 2527.8867924528304
           Else (feature 2 > 60.14999961853027)
            Predict: 2714.3333333333335
          Else (feature 7 not in {1.0,3.0,4.0,6.0})
           If (feature 9 in {0.0,1.0,2.0})
            Predict: 2643.56
           Else (feature 9 not in {0.0,1.0,2.0})
            Predict: 2897.5
        Else (feature 3 > 3001.5)
         If (feature 3 <= 3764.0)
          If (feature 3 <= 3383.0)
           If (feature 7 in {4.0,5.0})
            Predict: 3156.1052631578946
           Else (feature 7 not in {4.0,5.0})
            Predict: 3370.8
          Else (feature 3 > 3383.0)
           If (feature 1 <= 61.54999923706055)
            Predict: 3505.5
           Else (feature 1 > 61.54999923706055)
            Predict: 3450.75
         Else (feature 3 > 3764.0)
          If (feature 3 <= 4146.5)
           If (feature 1 <= 62.25)
            Predict: 3819.5
           Else (feature 1 > 62.25)
            Predict: 3941.0
          Else (feature 3 > 4146.5)
           If (feature 0 <= 0.5149999856948853)
            Predict: 4368.0
           Else (feature 0 > 0.5149999856948853)
            Predict: 4216.0
     Else (feature 0 > 0.574999988079071)
      If (feature 5 <= 5.605000019073486)
       If (feature 8 in {3.0,5.0,6.0})
        If (feature 8 in {5.0,6.0})
         If (feature 0 <= 0.6949999928474426)
          If (feature 3 <= 1584.0)
           If (feature 6 <= 3.3049999475479126)
            Predict: 1319.75
           Else (feature 6 > 3.3049999475479126)
            Predict: 1401.2307692307693
          Else (feature 3 > 1584.0)
           If (feature 1 <= 61.04999923706055)
            Predict: 1658.5714285714287
           Else (feature 1 > 61.04999923706055)
            Predict: 1840.6666666666667
         Else (feature 0 > 0.6949999928474426)
          If (feature 9 in {4.0})
           If (feature 2 <= 55.04999923706055)
            Predict: 1843.5
           Else (feature 2 > 55.04999923706055)
            Predict: 1697.0
          Else (feature 9 not in {4.0})
           If (feature 7 in {0.0})
            Predict: 1890.0
           Else (feature 7 not in {0.0})
            Predict: 2028.0
        Else (feature 8 not in {5.0,6.0})
         If (feature 3 <= 1940.5)
          If (feature 3 <= 1584.0)
           If (feature 5 <= 5.365000009536743)
            Predict: 1363.75
           Else (feature 5 > 5.365000009536743)
            Predict: 1439.6666666666667
          Else (feature 3 > 1584.0)
           If (feature 3 <= 1754.5)
            Predict: 1645.0588235294117
           Else (feature 3 > 1754.5)
            Predict: 1835.3666666666666
         Else (feature 3 > 1940.5)
          If (feature 3 <= 2186.0)
           If (feature 9 in {1.0,2.0})
            Predict: 1992.3333333333333
           Else (feature 9 not in {1.0,2.0})
            Predict: 2087.9
          Else (feature 3 > 2186.0)
           If (feature 9 in {0.0,1.0,3.0})
            Predict: 2297.681818181818
           Else (feature 9 not in {0.0,1.0,3.0})
            Predict: 2517.1666666666665
       Else (feature 8 not in {3.0,5.0,6.0})
        If (feature 7 in {0.0,1.0,2.0,3.0})
         If (feature 3 <= 1940.5)
          If (feature 5 <= 5.265000104904175)
           If (feature 1 <= 62.25)
            Predict: 1634.0
           Else (feature 1 > 62.25)
            Predict: 1404.1818181818182
          Else (feature 5 > 5.265000104904175)
           If (feature 6 <= 3.2350000143051147)
            Predict: 1437.0
           Else (feature 6 > 3.2350000143051147)
            Predict: 1691.3257328990228
         Else (feature 3 > 1940.5)
          If (feature 1 <= 62.45000076293945)
           If (feature 3 <= 2186.0)
            Predict: 2062.641304347826
           Else (feature 3 > 2186.0)
            Predict: 2367.512195121951
          Else (feature 1 > 62.45000076293945)
           If (feature 3 <= 2689.0)
            Predict: 2225.206896551724
           Else (feature 3 > 2689.0)
            Predict: 3138.0833333333335
        Else (feature 7 not in {0.0,1.0,2.0,3.0})
         If (feature 3 <= 3764.0)
          If (feature 3 <= 2689.0)
           If (feature 3 <= 2422.5)
            Predict: 2240.5365853658536
           Else (feature 3 > 2422.5)
            Predict: 2571.9166666666665
          Else (feature 3 > 2689.0)
           If (feature 2 <= 56.04999923706055)
            Predict: 3073.676470588235
           Else (feature 2 > 56.04999923706055)
            Predict: 2973.714285714286
         Else (feature 3 > 3764.0)
          If (feature 1 <= 62.35000038146973)
           If (feature 1 <= 62.14999961853027)
            Predict: 4395.625
           Else (feature 1 > 62.14999961853027)
            Predict: 3879.5
          Else (feature 1 > 62.35000038146973)
           Predict: 4996.0
      Else (feature 5 > 5.605000019073486)
       If (feature 3 <= 2689.0)
        If (feature 3 <= 2186.0)
         If (feature 8 in {1.0,5.0,6.0})
          If (feature 1 <= 62.64999961853027)
           If (feature 7 in {1.0,2.0})
            Predict: 1549.3333333333333
           Else (feature 7 not in {1.0,2.0})
            Predict: 1762.6666666666667
          Else (feature 1 > 62.64999961853027)
           If (feature 6 <= 3.5649999380111694)
            Predict: 1906.4285714285713
           Else (feature 6 > 3.5649999380111694)
            Predict: 2065.8571428571427
         Else (feature 8 not in {1.0,5.0,6.0})
          If (feature 3 <= 1940.5)
           If (feature 7 in {2.0})
            Predict: 1746.6
           Else (feature 7 not in {2.0})
            Predict: 1898.2
          Else (feature 3 > 1940.5)
           If (feature 1 <= 63.25)
            Predict: 2092.1481481481483
           Else (feature 1 > 63.25)
            Predict: 2069.8333333333335
        Else (feature 3 > 2186.0)
         If (feature 2 <= 58.04999923706055)
          If (feature 7 in {0.0,1.0,2.0,3.0,6.0})
           If (feature 3 <= 2422.5)
            Predict: 2342.3333333333335
           Else (feature 3 > 2422.5)
            Predict: 2479.6363636363635
          Else (feature 7 not in {0.0,1.0,2.0,3.0,6.0})
           If (feature 1 <= 63.04999923706055)
            Predict: 2575.8571428571427
           Else (feature 1 > 63.04999923706055)
            Predict: 2681.0
         Else (feature 2 > 58.04999923706055)
          If (feature 3 <= 2422.5)
           If (feature 1 <= 63.04999923706055)
            Predict: 2265.1428571428573
           Else (feature 1 > 63.04999923706055)
            Predict: 2238.5833333333335
          Else (feature 3 > 2422.5)
           If (feature 8 in {0.0})
            Predict: 2512.0
           Else (feature 8 not in {0.0})
            Predict: 2597.5
       Else (feature 3 > 2689.0)
        If (feature 7 in {1.0,3.0,4.0})
         If (feature 0 <= 0.6949999928474426)
          If (feature 9 in {2.0})
           Predict: 3050.0
          Else (feature 9 not in {2.0})
           Predict: 3103.0
         Else (feature 0 > 0.6949999928474426)
          If (feature 3 <= 3001.5)
           If (feature 2 <= 59.20000076293945)
            Predict: 2905.7272727272725
           Else (feature 2 > 59.20000076293945)
            Predict: 2812.0
          Else (feature 3 > 3001.5)
           If (feature 7 in {1.0})
            Predict: 3087.0
           Else (feature 7 not in {1.0})
            Predict: 3107.0
        Else (feature 7 not in {1.0,3.0,4.0})
         If (feature 9 in {2.0})
          If (feature 3 <= 3383.0)
           If (feature 0 <= 0.6949999928474426)
            Predict: 3309.0
           Else (feature 0 > 0.6949999928474426)
            Predict: 3261.0
          Else (feature 3 > 3383.0)
           If (feature 8 in {0.0})
            Predict: 3725.0
           Else (feature 8 not in {0.0})
            Predict: 4114.0
         Else (feature 9 not in {2.0})
          Predict: 5052.0
   Else (feature 4 > 5.605000019073486)
    If (feature 3 <= 3383.0)
     If (feature 3 <= 2689.0)
      If (feature 3 <= 2186.0)
       If (feature 3 <= 1754.5)
        If (feature 3 <= 1305.5)
         If (feature 0 <= 0.375)
          If (feature 0 <= 0.3449999988079071)
           Predict: 816.0
          Else (feature 0 > 0.3449999988079071)
           Predict: 949.0
         Else (feature 0 > 0.375)
          If (feature 2 <= 61.10000038146973)
           If (feature 0 <= 0.6949999928474426)
            Predict: 1250.8
           Else (feature 0 > 0.6949999928474426)
            Predict: 1151.5185185185185
          Else (feature 2 > 61.10000038146973)
           If (feature 8 in {0.0})
            Predict: 1047.0
           Else (feature 8 not in {0.0})
            Predict: 1123.75
        Else (feature 3 > 1305.5)
         If (feature 0 <= 0.6949999928474426)
          If (feature 8 in {0.0,2.0,5.0})
           If (feature 3 <= 1584.0)
            Predict: 1370.0
           Else (feature 3 > 1584.0)
            Predict: 1644.8
          Else (feature 8 not in {0.0,2.0,5.0})
           If (feature 5 <= 5.605000019073486)
            Predict: 1603.0
           Else (feature 5 > 5.605000019073486)
            Predict: 1708.0
         Else (feature 0 > 0.6949999928474426)
          If (feature 7 in {1.0,7.0})
           If (feature 1 <= 64.04999923706055)
            Predict: 1555.764705882353
           Else (feature 1 > 64.04999923706055)
            Predict: 1735.0
          Else (feature 7 not in {1.0,7.0})
           If (feature 1 <= 58.75)
            Predict: 1720.75
           Else (feature 1 > 58.75)
            Predict: 1645.1538461538462
       Else (feature 3 > 1754.5)
        If (feature 8 in {1.0,4.0,6.0})
         If (feature 0 <= 0.7150000035762787)
          If (feature 7 in {0.0,1.0,2.0,4.0})
           If (feature 2 <= 61.10000038146973)
            Predict: 1906.4204545454545
           Else (feature 2 > 61.10000038146973)
            Predict: 1981.4761904761904
          Else (feature 7 not in {0.0,1.0,2.0,4.0})
           If (feature 4 <= 5.704999923706055)
            Predict: 2011.625
           Else (feature 4 > 5.704999923706055)
            Predict: 2048.6
         Else (feature 0 > 0.7150000035762787)
          If (feature 8 in {6.0})
           If (feature 3 <= 1940.5)
            Predict: 1861.3243243243244
           Else (feature 3 > 1940.5)
            Predict: 2051.315789473684
          Else (feature 8 not in {6.0})
           If (feature 7 in {0.0})
            Predict: 2042.0
           Else (feature 7 not in {0.0})
            Predict: 2132.5833333333335
        Else (feature 8 not in {1.0,4.0,6.0})
         If (feature 1 <= 63.25)
          If (feature 1 <= 61.54999923706055)
           If (feature 3 <= 1940.5)
            Predict: 1848.061224489796
           Else (feature 3 > 1940.5)
            Predict: 2081.1463414634145
          Else (feature 1 > 61.54999923706055)
           If (feature 3 <= 1940.5)
            Predict: 1873.5
           Else (feature 3 > 1940.5)
            Predict: 2095.993006993007
         Else (feature 1 > 63.25)
          If (feature 6 <= 3.5649999380111694)
           If (feature 3 <= 1940.5)
            Predict: 1837.4444444444443
           Else (feature 3 > 1940.5)
            Predict: 2048.0
          Else (feature 6 > 3.5649999380111694)
           If (feature 7 in {2.0,7.0})
            Predict: 1951.392857142857
           Else (feature 7 not in {2.0,7.0})
            Predict: 2056.8260869565215
      Else (feature 3 > 2186.0)
       If (feature 8 in {0.0,2.0,3.0,5.0,6.0})
        If (feature 5 <= 5.775000095367432)
         If (feature 2 <= 58.04999923706055)
          If (feature 7 in {0.0,2.0,6.0})
           If (feature 1 <= 62.75)
            Predict: 2384.5766871165642
           Else (feature 1 > 62.75)
            Predict: 2315.846153846154
          Else (feature 7 not in {0.0,2.0,6.0})
           If (feature 4 <= 5.765000104904175)
            Predict: 2462.5510204081634
           Else (feature 4 > 5.765000104904175)
            Predict: 2558.32
         Else (feature 2 > 58.04999923706055)
          If (feature 3 <= 2422.5)
           If (feature 0 <= 0.7150000035762787)
            Predict: 2322.2235294117645
           Else (feature 0 > 0.7150000035762787)
            Predict: 2271.5365853658536
          Else (feature 3 > 2422.5)
           If (feature 8 in {3.0,5.0})
            Predict: 2488.846153846154
           Else (feature 8 not in {3.0,5.0})
            Predict: 2544.4545454545455
        Else (feature 5 > 5.775000095367432)
         If (feature 8 in {6.0})
          If (feature 2 <= 56.04999923706055)
           If (feature 3 <= 2422.5)
            Predict: 2323.125
           Else (feature 3 > 2422.5)
            Predict: 2467.222222222222
          Else (feature 2 > 56.04999923706055)
           If (feature 9 in {0.0,1.0,3.0})
            Predict: 2293.5384615384614
           Else (feature 9 not in {0.0,1.0,3.0})
            Predict: 2368.4666666666667
         Else (feature 8 not in {6.0})
          If (feature 0 <= 0.7150000035762787)
           If (feature 5 <= 5.855000019073486)
            Predict: 2433.4210526315787
           Else (feature 5 > 5.855000019073486)
            Predict: 2369.714285714286
          Else (feature 0 > 0.7150000035762787)
           If (feature 3 <= 2422.5)
            Predict: 2329.1582733812947
           Else (feature 3 > 2422.5)
            Predict: 2555.276752767528
       Else (feature 8 not in {0.0,2.0,3.0,5.0,6.0})
        If (feature 3 <= 2422.5)
         If (feature 9 in {2.0})
          If (feature 0 <= 0.6949999928474426)
           Predict: 2227.0
          Else (feature 0 > 0.6949999928474426)
           If (feature 5 <= 5.775000095367432)
            Predict: 2293.6
           Else (feature 5 > 5.775000095367432)
            Predict: 2336.625
         Else (feature 9 not in {2.0})
          If (feature 1 <= 58.75)
           If (feature 5 <= 5.855000019073486)
            Predict: 2297.8571428571427
           Else (feature 5 > 5.855000019073486)
            Predict: 2361.0
          Else (feature 1 > 58.75)
           If (feature 8 in {4.0})
            Predict: 2332.560975609756
           Else (feature 8 not in {4.0})
            Predict: 2351.5692307692307
        Else (feature 3 > 2422.5)
         If (feature 7 in {2.0,7.0})
          If (feature 4 <= 5.8450000286102295)
           If (feature 1 <= 61.85000038146973)
            Predict: 2564.75
           Else (feature 1 > 61.85000038146973)
            Predict: 2513.2
          Else (feature 4 > 5.8450000286102295)
           If (feature 2 <= 55.04999923706055)
            Predict: 2652.5714285714284
           Else (feature 2 > 55.04999923706055)
            Predict: 2573.3684210526317
         Else (feature 7 not in {2.0,7.0})
          If (feature 6 <= 3.4450000524520874)
           If (feature 7 in {1.0})
            Predict: 2473.0
           Else (feature 7 not in {1.0})
            Predict: 2546.4
          Else (feature 6 > 3.4450000524520874)
           If (feature 5 <= 5.7149999141693115)
            Predict: 2574.943661971831
           Else (feature 5 > 5.7149999141693115)
            Predict: 2604.5975609756097
     Else (feature 3 > 2689.0)
      If (feature 5 <= 6.025000095367432)
       If (feature 2 <= 57.04999923706055)
        If (feature 3 <= 3001.5)
         If (feature 1 <= 62.45000076293945)
          If (feature 8 in {4.0,6.0})
           If (feature 4 <= 5.8450000286102295)
            Predict: 2826.2156862745096
           Else (feature 4 > 5.8450000286102295)
            Predict: 2767.3333333333335
          Else (feature 8 not in {4.0,6.0})
           If (feature 6 <= 3.5649999380111694)
            Predict: 2848.4615384615386
           Else (feature 6 > 3.5649999380111694)
            Predict: 2865.75
         Else (feature 1 > 62.45000076293945)
          If (feature 7 in {0.0,1.0,4.0})
           If (feature 7 in {4.0})
            Predict: 2694.0
           Else (feature 7 not in {4.0})
            Predict: 2807.223214285714
          Else (feature 7 not in {0.0,1.0,4.0})
           If (feature 8 in {0.0,1.0})
            Predict: 2802.1111111111113
           Else (feature 8 not in {0.0,1.0})
            Predict: 2889.1923076923076
        Else (feature 3 > 3001.5)
         If (feature 7 in {0.0,2.0})
          If (feature 8 in {0.0,1.0,3.0,4.0})
           If (feature 9 in {1.0,2.0,4.0})
            Predict: 3096.259259259259
           Else (feature 9 not in {1.0,2.0,4.0})
            Predict: 3148.864864864865
          Else (feature 8 not in {0.0,1.0,3.0,4.0})
           If (feature 1 <= 61.45000076293945)
            Predict: 3151.875
           Else (feature 1 > 61.45000076293945)
            Predict: 3267.909090909091
         Else (feature 7 not in {0.0,2.0})
          If (feature 6 <= 3.5649999380111694)
           If (feature 8 in {2.0,3.0,4.0,5.0})
            Predict: 3150.294117647059
           Else (feature 8 not in {2.0,3.0,4.0,5.0})
            Predict: 3197.328767123288
          Else (feature 6 > 3.5649999380111694)
           If (feature 8 in {0.0,2.0,3.0,5.0})
            Predict: 3191.3513513513512
           Else (feature 8 not in {0.0,2.0,3.0,5.0})
            Predict: 3253.1428571428573
       Else (feature 2 > 57.04999923706055)
        If (feature 6 <= 3.725000023841858)
         If (feature 3 <= 3001.5)
          If (feature 2 <= 61.10000038146973)
           If (feature 7 in {0.0,1.0,2.0,5.0})
            Predict: 2823.1962264150943
           Else (feature 7 not in {0.0,1.0,2.0,5.0})
            Predict: 2875.0
          Else (feature 2 > 61.10000038146973)
           If (feature 8 in {0.0,2.0,3.0})
            Predict: 2774.8260869565215
           Else (feature 8 not in {0.0,2.0,3.0})
            Predict: 2846.9444444444443
         Else (feature 3 > 3001.5)
          If (feature 2 <= 58.04999923706055)
           If (feature 5 <= 5.855000019073486)
            Predict: 3162.808510638298
           Else (feature 5 > 5.855000019073486)
            Predict: 3236.0384615384614
          Else (feature 2 > 58.04999923706055)
           If (feature 7 in {0.0,1.0,2.0})
            Predict: 3104.903846153846
           Else (feature 7 not in {0.0,1.0,2.0})
            Predict: 3169.157894736842
        Else (feature 6 > 3.725000023841858)
         If (feature 4 <= 5.8450000286102295)
          If (feature 8 in {0.0,5.0})
           If (feature 8 in {5.0})
            Predict: 2714.0
           Else (feature 8 not in {5.0})
            Predict: 2753.0
          Else (feature 8 not in {0.0,5.0})
           Predict: 2941.0
         Else (feature 4 > 5.8450000286102295)
          If (feature 3 <= 3001.5)
           If (feature 8 in {1.0,5.0,6.0})
            Predict: 2788.6923076923076
           Else (feature 8 not in {1.0,5.0,6.0})
            Predict: 2882.9473684210525
          Else (feature 3 > 3001.5)
           If (feature 2 <= 58.04999923706055)
            Predict: 3140.076923076923
           Else (feature 2 > 58.04999923706055)
            Predict: 3226.7
      Else (feature 5 > 6.025000095367432)
       If (feature 1 <= 61.95000076293945)
        If (feature 3 <= 3001.5)
         If (feature 6 <= 3.6149998903274536)
          If (feature 9 in {1.0,4.0})
           If (feature 8 in {3.0})
            Predict: 2743.0
           Else (feature 8 not in {3.0})
            Predict: 2879.375
          Else (feature 9 not in {1.0,4.0})
           If (feature 7 in {1.0,2.0})
            Predict: 2899.75
           Else (feature 7 not in {1.0,2.0})
            Predict: 2954.6
         Else (feature 6 > 3.6149998903274536)
          If (feature 4 <= 6.025000095367432)
           If (feature 2 <= 55.04999923706055)
            Predict: 2938.5
           Else (feature 2 > 55.04999923706055)
            Predict: 2743.0
          Else (feature 4 > 6.025000095367432)
           If (feature 8 in {0.0,4.0,5.0,6.0})
            Predict: 2815.4761904761904
           Else (feature 8 not in {0.0,4.0,5.0,6.0})
            Predict: 2885.1052631578946
        Else (feature 3 > 3001.5)
         If (feature 7 in {0.0,3.0,5.0})
          If (feature 2 <= 57.04999923706055)
           If (feature 1 <= 61.45000076293945)
            Predict: 3222.25
           Else (feature 1 > 61.45000076293945)
            Predict: 3313.25
          Else (feature 2 > 57.04999923706055)
           If (feature 8 in {2.0})
            Predict: 3101.0
           Else (feature 8 not in {2.0})
            Predict: 3175.6315789473683
         Else (feature 7 not in {0.0,3.0,5.0})
          If (feature 9 in {0.0,2.0,3.0,4.0})
           If (feature 5 <= 6.174999952316284)
            Predict: 3198.076923076923
           Else (feature 5 > 6.174999952316284)
            Predict: 3253.222222222222
          Else (feature 9 not in {0.0,2.0,3.0,4.0})
           If (feature 2 <= 60.14999961853027)
            Predict: 3311.3333333333335
           Else (feature 2 > 60.14999961853027)
            Predict: 3268.25
       Else (feature 1 > 61.95000076293945)
        If (feature 9 in {0.0,4.0})
         If (feature 8 in {0.0,3.0,4.0,5.0,6.0})
          If (feature 1 <= 62.35000038146973)
           If (feature 1 <= 62.14999961853027)
            Predict: 3022.0
           Else (feature 1 > 62.14999961853027)
            Predict: 2824.714285714286
          Else (feature 1 > 62.35000038146973)
           If (feature 7 in {1.0,2.0})
            Predict: 2965.4615384615386
           Else (feature 7 not in {1.0,2.0})
            Predict: 3140.8333333333335
         Else (feature 8 not in {0.0,3.0,4.0,5.0,6.0})
          If (feature 3 <= 3001.5)
           Predict: 2790.0
          Else (feature 3 > 3001.5)
           If (feature 9 in {4.0})
            Predict: 3236.0
           Else (feature 9 not in {4.0})
            Predict: 3317.0
        Else (feature 9 not in {0.0,4.0})
         If (feature 7 in {7.0})
          Predict: 2698.0
         Else (feature 7 not in {7.0})
          If (feature 8 in {1.0,5.0,6.0})
           If (feature 3 <= 3001.5)
            Predict: 2912.8333333333335
           Else (feature 3 > 3001.5)
            Predict: 3204.183673469388
          Else (feature 8 not in {1.0,5.0,6.0})
           If (feature 4 <= 6.025000095367432)
            Predict: 3089.3333333333335
           Else (feature 4 > 6.025000095367432)
            Predict: 3232.487804878049
    Else (feature 3 > 3383.0)
     If (feature 5 <= 6.025000095367432)
      If (feature 0 <= 0.9050000011920929)
       If (feature 3 <= 4146.5)
        If (feature 3 <= 3764.0)
         If (feature 9 in {1.0,4.0})
          If (feature 2 <= 58.04999923706055)
           If (feature 7 in {1.0,3.0})
            Predict: 3521.0
           Else (feature 7 not in {1.0,3.0})
            Predict: 3626.777777777778
          Else (feature 2 > 58.04999923706055)
           If (feature 7 in {1.0,2.0,3.0,6.0})
            Predict: 3448.2272727272725
           Else (feature 7 not in {1.0,2.0,3.0,6.0})
            Predict: 3621.0
         Else (feature 9 not in {1.0,4.0})
          If (feature 4 <= 5.704999923706055)
           If (feature 1 <= 61.64999961853027)
            Predict: 3647.6315789473683
           Else (feature 1 > 61.64999961853027)
            Predict: 3551.103448275862
          Else (feature 4 > 5.704999923706055)
           If (feature 8 in {0.0,1.0,3.0,4.0})
            Predict: 3544.957219251337
           Else (feature 8 not in {0.0,1.0,3.0,4.0})
            Predict: 3591.135135135135
        Else (feature 3 > 3764.0)
         If (feature 7 in {0.0,3.0,4.0,5.0})
          If (feature 0 <= 0.7150000035762787)
           If (feature 5 <= 5.7149999141693115)
            Predict: 3860.0
           Else (feature 5 > 5.7149999141693115)
            Predict: 3985.176470588235
          Else (feature 0 > 0.7150000035762787)
           If (feature 6 <= 3.6149998903274536)
            Predict: 3847.818181818182
           Else (feature 6 > 3.6149998903274536)
            Predict: 3948.4444444444443
         Else (feature 7 not in {0.0,3.0,4.0,5.0})
          If (feature 6 <= 3.84499990940094)
           If (feature 9 in {3.0})
            Predict: 3792.0
           Else (feature 9 not in {3.0})
            Predict: 3996.769230769231
          Else (feature 6 > 3.84499990940094)
           Predict: 4086.1666666666665
       Else (feature 3 > 4146.5)
        If (feature 7 in {1.0,3.0,6.0})
         If (feature 3 <= 4455.5)
          If (feature 2 <= 55.04999923706055)
           If (feature 1 <= 60.25)
            Predict: 4444.0
           Else (feature 1 > 60.25)
            Predict: 4358.714285714285
          Else (feature 2 > 55.04999923706055)
           If (feature 6 <= 3.5649999380111694)
            Predict: 4351.5
           Else (feature 6 > 3.5649999380111694)
            Predict: 4244.0
         Else (feature 3 > 4455.5)
          If (feature 3 <= 4820.0)
           If (feature 2 <= 54.14999961853027)
            Predict: 4580.333333333333
           Else (feature 2 > 54.14999961853027)
            Predict: 4733.8
          Else (feature 3 > 4820.0)
           Predict: 5007.0
        Else (feature 7 not in {1.0,3.0,6.0})
         If (feature 9 in {1.0,2.0})
          If (feature 2 <= 57.04999923706055)
           If (feature 3 <= 4820.0)
            Predict: 4322.0
           Else (feature 3 > 4820.0)
            Predict: 4952.5
          Else (feature 2 > 57.04999923706055)
           If (feature 3 <= 4455.5)
            Predict: 4276.142857142857
           Else (feature 3 > 4455.5)
            Predict: 4592.0
         Else (feature 9 not in {1.0,2.0})
          If (feature 3 <= 4820.0)
           If (feature 2 <= 55.04999923706055)
            Predict: 4524.0
           Else (feature 2 > 55.04999923706055)
            Predict: 4324.0
          Else (feature 3 > 4820.0)
           If (feature 4 <= 5.8450000286102295)
            Predict: 5214.695652173913
           Else (feature 4 > 5.8450000286102295)
            Predict: 5862.4
      Else (feature 0 > 0.9050000011920929)
       If (feature 1 <= 63.25)
        Predict: 5430.0
       Else (feature 1 > 63.25)
        If (feature 7 in {2.0})
         Predict: 3384.0
        Else (feature 7 not in {2.0})
         If (feature 1 <= 66.54999923706055)
          If (feature 4 <= 6.025000095367432)
           Predict: 4439.0
          Else (feature 4 > 6.025000095367432)
           Predict: 4107.0
         Else (feature 1 > 66.54999923706055)
          Predict: 4482.0
     Else (feature 5 > 6.025000095367432)
      If (feature 8 in {5.0,6.0})
       If (feature 3 <= 4146.5)
        If (feature 0 <= 0.9050000011920929)
         If (feature 7 in {0.0,1.0,3.0,4.0,6.0})
          If (feature 3 <= 3764.0)
           If (feature 7 in {0.0,1.0})
            Predict: 3489.75
           Else (feature 7 not in {0.0,1.0})
            Predict: 3633.5833333333335
          Else (feature 3 > 3764.0)
           If (feature 9 in {0.0})
            Predict: 3873.6666666666665
           Else (feature 9 not in {0.0})
            Predict: 3998.2
         Else (feature 7 not in {0.0,1.0,3.0,4.0,6.0})
          If (feature 5 <= 6.174999952316284)
           If (feature 9 in {3.0})
            Predict: 3741.0
           Else (feature 9 not in {3.0})
            Predict: 3823.0
          Else (feature 5 > 6.174999952316284)
           If (feature 1 <= 60.54999923706055)
            Predict: 3746.0
           Else (feature 1 > 60.54999923706055)
            Predict: 4099.0
        Else (feature 0 > 0.9050000011920929)
         If (feature 3 <= 3764.0)
          If (feature 7 in {0.0,4.0})
           If (feature 2 <= 60.14999961853027)
            Predict: 3635.5
           Else (feature 2 > 60.14999961853027)
            Predict: 3544.0
          Else (feature 7 not in {0.0,4.0})
           If (feature 5 <= 6.384999990463257)
            Predict: 3706.5454545454545
           Else (feature 5 > 6.384999990463257)
            Predict: 3462.0
         Else (feature 3 > 3764.0)
          If (feature 9 in {0.0,1.0,3.0})
           If (feature 4 <= 6.384999990463257)
            Predict: 3901.266666666667
           Else (feature 4 > 6.384999990463257)
            Predict: 3808.0
          Else (feature 9 not in {0.0,1.0,3.0})
           If (feature 4 <= 6.315000057220459)
            Predict: 3994.0
           Else (feature 4 > 6.315000057220459)
            Predict: 4002.0
       Else (feature 3 > 4146.5)
        If (feature 4 <= 6.174999952316284)
         If (feature 2 <= 58.04999923706055)
          If (feature 0 <= 0.9050000011920929)
           Predict: 4161.0
          Else (feature 0 > 0.9050000011920929)
           Predict: 4199.0
         Else (feature 2 > 58.04999923706055)
          If (feature 5 <= 6.174999952316284)
           Predict: 4516.0
          Else (feature 5 > 6.174999952316284)
           Predict: 4355.0
        Else (feature 4 > 6.174999952316284)
         If (feature 7 in {3.0})
          Predict: 4587.0
         Else (feature 7 not in {3.0})
          If (feature 4 <= 6.315000057220459)
           If (feature 1 <= 60.25)
            Predict: 4609.0
           Else (feature 1 > 60.25)
            Predict: 4488.0
          Else (feature 4 > 6.315000057220459)
           Predict: 4763.0
      Else (feature 8 not in {5.0,6.0})
       If (feature 7 in {0.0,1.0,2.0,3.0})
        If (feature 3 <= 4455.5)
         If (feature 3 <= 3764.0)
          If (feature 7 in {1.0,2.0,3.0})
           If (feature 2 <= 60.14999961853027)
            Predict: 3603.3699421965316
           Else (feature 2 > 60.14999961853027)
            Predict: 3520.542857142857
          Else (feature 7 not in {1.0,2.0,3.0})
           If (feature 2 <= 56.04999923706055)
            Predict: 3560.7894736842104
           Else (feature 2 > 56.04999923706055)
            Predict: 3639.7830188679245
         Else (feature 3 > 3764.0)
          If (feature 3 <= 4146.5)
           If (feature 4 <= 6.174999952316284)
            Predict: 3950.5424528301887
           Else (feature 4 > 6.174999952316284)
            Predict: 3984.855
          Else (feature 3 > 4146.5)
           If (feature 1 <= 61.85000038146973)
            Predict: 4271.723076923077
           Else (feature 1 > 61.85000038146973)
            Predict: 4304.674698795181
        Else (feature 3 > 4455.5)
         If (feature 9 in {1.0,2.0,3.0,4.0})
          If (feature 1 <= 63.04999923706055)
           If (feature 5 <= 6.454999923706055)
            Predict: 4834.289340101523
           Else (feature 5 > 6.454999923706055)
            Predict: 5997.0
          Else (feature 1 > 63.04999923706055)
           If (feature 4 <= 6.174999952316284)
            Predict: 4596.8378378378375
           Else (feature 4 > 6.174999952316284)
            Predict: 4793.75
         Else (feature 9 not in {1.0,2.0,3.0,4.0})
          If (feature 7 in {0.0,1.0,2.0})
           If (feature 7 in {2.0})
            Predict: 4757.117647058823
           Else (feature 7 not in {2.0})
            Predict: 5086.265625
          Else (feature 7 not in {0.0,1.0,2.0})
           If (feature 3 <= 5201.5)
            Predict: 4833.166666666667
           Else (feature 3 > 5201.5)
            Predict: 5869.875
       Else (feature 7 not in {0.0,1.0,2.0,3.0})
        If (feature 8 in {0.0,3.0})
         If (feature 3 <= 5201.5)
          If (feature 3 <= 4455.5)
           If (feature 0 <= 0.8349999785423279)
            Predict: 3781.0
           Else (feature 0 > 0.8349999785423279)
            Predict: 4345.6
          Else (feature 3 > 4455.5)
           If (feature 2 <= 57.04999923706055)
            Predict: 5183.0
           Else (feature 2 > 57.04999923706055)
            Predict: 4672.347826086957
         Else (feature 3 > 5201.5)
          If (feature 4 <= 6.384999990463257)
           If (feature 9 in {1.0,3.0})
            Predict: 5489.875
           Else (feature 9 not in {1.0,3.0})
            Predict: 5716.785714285715
          Else (feature 4 > 6.384999990463257)
           Predict: 6801.0
        Else (feature 8 not in {0.0,3.0})
         If (feature 3 <= 5716.5)
          If (feature 3 <= 4455.5)
           Predict: 4243.0
          Else (feature 3 > 4455.5)
           If (feature 6 <= 3.84499990940094)
            Predict: 5527.5
           Else (feature 6 > 3.84499990940094)
            Predict: 5078.0
         Else (feature 3 > 5716.5)
          If (feature 8 in {1.0,2.0})
           If (feature 1 <= 62.25)
            Predict: 6664.875
           Else (feature 1 > 62.25)
            Predict: 7347.25
          Else (feature 8 not in {1.0,2.0})
           If (feature 4 <= 6.174999952316284)
            Predict: 7642.666666666667
           Else (feature 4 > 6.174999952316284)
            Predict: 8707.0
  Else (feature 0 > 0.9950000047683716)
   If (feature 3 <= 9692.5)
    If (feature 4 <= 7.015000104904175)
     If (feature 3 <= 5716.5)
      If (feature 9 in {3.0,4.0})
       If (feature 7 in {7.0})
        If (feature 4 <= 6.724999904632568)
         If (feature 3 <= 2689.0)
          If (feature 0 <= 1.1449999809265137)
           If (feature 3 <= 2186.0)
            Predict: 2034.5
           Else (feature 3 > 2186.0)
            Predict: 2300.9333333333334
          Else (feature 0 > 1.1449999809265137)
           If (feature 8 in {0.0})
            Predict: 2376.0
           Else (feature 8 not in {0.0})
            Predict: 2622.0
         Else (feature 3 > 2689.0)
          If (feature 3 <= 3001.5)
           If (feature 4 <= 6.315000057220459)
            Predict: 2749.8571428571427
           Else (feature 4 > 6.315000057220459)
            Predict: 2960.6
          Else (feature 3 > 3001.5)
           If (feature 3 <= 3383.0)
            Predict: 3152.5
           Else (feature 3 > 3383.0)
            Predict: 3564.6666666666665
        Else (feature 4 > 6.724999904632568)
         If (feature 3 <= 3764.0)
          If (feature 3 <= 3001.5)
           If (feature 0 <= 1.215000033378601)
            Predict: 2757.6666666666665
           Else (feature 0 > 1.215000033378601)
            Predict: 2512.0
          Else (feature 3 > 3001.5)
           If (feature 8 in {1.0,3.0,5.0})
            Predict: 3142.25
           Else (feature 8 not in {1.0,3.0,5.0})
            Predict: 3415.0
         Else (feature 3 > 3764.0)
          If (feature 6 <= 4.515000104904175)
           If (feature 0 <= 1.2949999570846558)
            Predict: 4277.0
           Else (feature 0 > 1.2949999570846558)
            Predict: 4140.0
          Else (feature 6 > 4.515000104904175)
           If (feature 2 <= 56.04999923706055)
            Predict: 4862.666666666667
           Else (feature 2 > 56.04999923706055)
            Predict: 4495.428571428572
       Else (feature 7 not in {7.0})
        If (feature 3 <= 4455.5)
         If (feature 8 in {0.0,5.0,6.0})
          If (feature 3 <= 3764.0)
           If (feature 3 <= 3383.0)
            Predict: 3065.181818181818
           Else (feature 3 > 3383.0)
            Predict: 3587.9811320754716
          Else (feature 3 > 3764.0)
           If (feature 8 in {6.0})
            Predict: 3925.2
           Else (feature 8 not in {6.0})
            Predict: 4041.7912087912086
         Else (feature 8 not in {0.0,5.0,6.0})
          If (feature 3 <= 3764.0)
           If (feature 3 <= 3383.0)
            Predict: 3051.1428571428573
           Else (feature 3 > 3383.0)
            Predict: 3645.1290322580644
          Else (feature 3 > 3764.0)
           If (feature 6 <= 3.725000023841858)
            Predict: 3892.3333333333335
           Else (feature 6 > 3.725000023841858)
            Predict: 4162.657276995305
        Else (feature 3 > 4455.5)
         If (feature 5 <= 6.025000095367432)
          If (feature 6 <= 4.144999980926514)
           Predict: 5666.0
          Else (feature 6 > 4.144999980926514)
           Predict: 5294.0
         Else (feature 5 > 6.025000095367432)
          If (feature 7 in {0.0,2.0,4.0})
           If (feature 8 in {0.0,2.0,3.0,5.0})
            Predict: 4865.374193548387
           Else (feature 8 not in {0.0,2.0,3.0,5.0})
            Predict: 5011.72027972028
          Else (feature 7 not in {0.0,2.0,4.0})
           If (feature 8 in {1.0,3.0,5.0,6.0})
            Predict: 4982.69014084507
           Else (feature 8 not in {1.0,3.0,5.0,6.0})
            Predict: 5317.387096774193
      Else (feature 9 not in {3.0,4.0})
       If (feature 5 <= 6.7149999141693115)
        If (feature 3 <= 4455.5)
         If (feature 8 in {3.0,5.0,6.0})
          If (feature 3 <= 3764.0)
           If (feature 4 <= 6.384999990463257)
            Predict: 3543.095238095238
           Else (feature 4 > 6.384999990463257)
            Predict: 3318.1420118343194
          Else (feature 3 > 3764.0)
           If (feature 9 in {1.0})
            Predict: 4093.194285714286
           Else (feature 9 not in {1.0})
            Predict: 4195.677824267783
         Else (feature 8 not in {3.0,5.0,6.0})
          If (feature 1 <= 61.54999923706055)
           If (feature 7 in {7.0})
            Predict: 3378.6969696969695
           Else (feature 7 not in {7.0})
            Predict: 4111.3015873015875
          Else (feature 1 > 61.54999923706055)
           If (feature 3 <= 4146.5)
            Predict: 3780.5524475524476
           Else (feature 3 > 4146.5)
            Predict: 4321.115384615385
        Else (feature 3 > 4455.5)
         If (feature 3 <= 5201.5)
          If (feature 2 <= 59.20000076293945)
           If (feature 7 in {2.0})
            Predict: 4742.415422885572
           Else (feature 7 not in {2.0})
            Predict: 4859.485294117647
          Else (feature 2 > 59.20000076293945)
           If (feature 5 <= 6.454999923706055)
            Predict: 4716.955555555555
           Else (feature 5 > 6.454999923706055)
            Predict: 4809.847457627118
         Else (feature 3 > 5201.5)
          If (feature 8 in {0.0,3.0,5.0,6.0})
           If (feature 6 <= 4.144999980926514)
            Predict: 5410.772908366534
           Else (feature 6 > 4.144999980926514)
            Predict: 5515.0
          Else (feature 8 not in {0.0,3.0,5.0,6.0})
           If (feature 1 <= 61.95000076293945)
            Predict: 5433.934523809524
           Else (feature 1 > 61.95000076293945)
            Predict: 5493.968
       Else (feature 5 > 6.7149999141693115)
        If (feature 3 <= 4820.0)
         If (feature 7 in {7.0})
          If (feature 0 <= 1.2949999570846558)
           If (feature 1 <= 62.35000038146973)
            Predict: 3735.5227272727275
           Else (feature 1 > 62.35000038146973)
            Predict: 2360.0
          Else (feature 0 > 1.2949999570846558)
           Predict: 4560.0
         Else (feature 7 not in {7.0})
          If (feature 2 <= 58.04999923706055)
           If (feature 3 <= 4455.5)
            Predict: 4172.038461538462
           Else (feature 3 > 4455.5)
            Predict: 4654.745762711865
          Else (feature 2 > 58.04999923706055)
           If (feature 6 <= 4.075000047683716)
            Predict: 4199.941176470588
           Else (feature 6 > 4.075000047683716)
            Predict: 4488.479166666667
        Else (feature 3 > 4820.0)
         If (feature 8 in {1.0,2.0,6.0})
          If (feature 7 in {0.0,1.0,2.0})
           If (feature 2 <= 56.04999923706055)
            Predict: 5239.821428571428
           Else (feature 2 > 56.04999923706055)
            Predict: 5158.186440677966
          Else (feature 7 not in {0.0,1.0,2.0})
           If (feature 3 <= 5201.5)
            Predict: 5035.454545454545
           Else (feature 3 > 5201.5)
            Predict: 5525.772727272727
         Else (feature 8 not in {1.0,2.0,6.0})
          If (feature 3 <= 5201.5)
           If (feature 1 <= 62.85000038146973)
            Predict: 5027.214285714285
           Else (feature 1 > 62.85000038146973)
            Predict: 5192.0
          Else (feature 3 > 5201.5)
           If (feature 1 <= 61.35000038146973)
            Predict: 5511.333333333333
           Else (feature 1 > 61.35000038146973)
            Predict: 5450.6122448979595
     Else (feature 3 > 5716.5)
      If (feature 3 <= 7236.0)
       If (feature 9 in {1.0,4.0})
        If (feature 7 in {0.0,2.0})
         If (feature 0 <= 1.215000033378601)
          If (feature 4 <= 6.724999904632568)
           If (feature 6 <= 4.075000047683716)
            Predict: 5852.318181818182
           Else (feature 6 > 4.075000047683716)
            Predict: 6143.8
          Else (feature 4 > 6.724999904632568)
           If (feature 3 <= 6376.5)
            Predict: 5963.672727272728
           Else (feature 3 > 6376.5)
            Predict: 6652.25
         Else (feature 0 > 1.215000033378601)
          If (feature 1 <= 61.95000076293945)
           If (feature 3 <= 6376.5)
            Predict: 6238.5
           Else (feature 3 > 6376.5)
            Predict: 6843.464285714285
          Else (feature 1 > 61.95000076293945)
           If (feature 2 <= 59.20000076293945)
            Predict: 6433.153846153846
           Else (feature 2 > 59.20000076293945)
            Predict: 5984.0
        Else (feature 7 not in {0.0,2.0})
         If (feature 2 <= 53.95000076293945)
          Predict: 6854.0
         Else (feature 2 > 53.95000076293945)
          If (feature 3 <= 6376.5)
           If (feature 0 <= 1.0149999856948853)
            Predict: 6099.694736842105
           Else (feature 0 > 1.0149999856948853)
            Predict: 6046.0625
          Else (feature 3 > 6376.5)
           If (feature 4 <= 6.525000095367432)
            Predict: 6700.867346938776
           Else (feature 4 > 6.525000095367432)
            Predict: 6762.529411764706
       Else (feature 9 not in {1.0,4.0})
        If (feature 8 in {2.0,3.0,4.0,5.0,6.0})
         If (feature 7 in {0.0,1.0,2.0})
          If (feature 8 in {5.0,6.0})
           If (feature 6 <= 4.144999980926514)
            Predict: 5905.555555555556
           Else (feature 6 > 4.144999980926514)
            Predict: 6142.961038961039
          Else (feature 8 not in {5.0,6.0})
           If (feature 1 <= 63.54999923706055)
            Predict: 6337.422018348624
           Else (feature 1 > 63.54999923706055)
            Predict: 6467.944444444444
         Else (feature 7 not in {0.0,1.0,2.0})
          If (feature 2 <= 56.04999923706055)
           If (feature 3 <= 6376.5)
            Predict: 6101.0625
           Else (feature 3 > 6376.5)
            Predict: 6875.569230769231
          Else (feature 2 > 56.04999923706055)
           If (feature 3 <= 6376.5)
            Predict: 6055.395833333333
           Else (feature 3 > 6376.5)
            Predict: 6798.152542372882
        Else (feature 8 not in {2.0,3.0,4.0,5.0,6.0})
         If (feature 5 <= 6.525000095367432)
          If (feature 1 <= 62.85000038146973)
           If (feature 3 <= 6376.5)
            Predict: 6111.126582278481
           Else (feature 3 > 6376.5)
            Predict: 6772.165137614679
          Else (feature 1 > 62.85000038146973)
           If (feature 3 <= 6376.5)
            Predict: 6024.768115942029
           Else (feature 3 > 6376.5)
            Predict: 6732.273972602739
         Else (feature 5 > 6.525000095367432)
          If (feature 0 <= 1.0049999952316284)
           If (feature 1 <= 58.75)
            Predict: 5921.0
           Else (feature 1 > 58.75)
            Predict: 5766.0
          Else (feature 0 > 1.0049999952316284)
           If (feature 4 <= 6.605000019073486)
            Predict: 6722.105263157895
           Else (feature 4 > 6.605000019073486)
            Predict: 6572.597484276729
      Else (feature 3 > 7236.0)
       If (feature 4 <= 6.724999904632568)
        If (feature 8 in {0.0,1.0,3.0,4.0})
         If (feature 3 <= 8454.5)
          If (feature 8 in {1.0,3.0})
           If (feature 4 <= 6.605000019073486)
            Predict: 7589.710526315789
           Else (feature 4 > 6.605000019073486)
            Predict: 7900.24
          Else (feature 8 not in {1.0,3.0})
           If (feature 2 <= 55.04999923706055)
            Predict: 7936.115384615385
           Else (feature 2 > 55.04999923706055)
            Predict: 7769.20814479638
         Else (feature 3 > 8454.5)
          If (feature 4 <= 6.525000095367432)
           If (feature 0 <= 1.0049999952316284)
            Predict: 9187.57142857143
           Else (feature 0 > 1.0049999952316284)
            Predict: 8783.53488372093
          Else (feature 4 > 6.525000095367432)
           If (feature 1 <= 61.75)
            Predict: 9126.783783783783
           Else (feature 1 > 61.75)
            Predict: 8942.130434782608
        Else (feature 8 not in {0.0,1.0,3.0,4.0})
         If (feature 6 <= 4.005000114440918)
          If (feature 3 <= 8454.5)
           If (feature 0 <= 1.0049999952316284)
            Predict: 8077.714285714285
           Else (feature 0 > 1.0049999952316284)
            Predict: 7727.482758620689
          Else (feature 3 > 8454.5)
           If (feature 7 in {3.0,4.0})
            Predict: 8803.07142857143
           Else (feature 7 not in {3.0,4.0})
            Predict: 9598.0
         Else (feature 6 > 4.005000114440918)
          If (feature 7 in {1.0,3.0})
           If (feature 9 in {1.0,2.0})
            Predict: 7735.125
           Else (feature 9 not in {1.0,2.0})
            Predict: 8188.935483870968
          Else (feature 7 not in {1.0,3.0})
           If (feature 3 <= 8454.5)
            Predict: 8112.5
           Else (feature 3 > 8454.5)
            Predict: 9180.5
       Else (feature 4 > 6.724999904632568)
        If (feature 7 in {0.0,1.0,2.0,3.0,6.0})
         If (feature 3 <= 8454.5)
          If (feature 8 in {3.0,4.0,5.0})
           If (feature 1 <= 62.14999961853027)
            Predict: 7831.2
           Else (feature 1 > 62.14999961853027)
            Predict: 7559.111111111111
          Else (feature 8 not in {3.0,4.0,5.0})
           If (feature 6 <= 4.0350000858306885)
            Predict: 7483.5
           Else (feature 6 > 4.0350000858306885)
            Predict: 7912.473282442748
         Else (feature 3 > 8454.5)
          If (feature 1 <= 62.04999923706055)
           If (feature 6 <= 4.315000057220459)
            Predict: 9097.733944954129
           Else (feature 6 > 4.315000057220459)
            Predict: 9410.6
          Else (feature 1 > 62.04999923706055)
           If (feature 2 <= 53.95000076293945)
            Predict: 9257.0
           Else (feature 2 > 53.95000076293945)
            Predict: 8803.475409836066
        Else (feature 7 not in {0.0,1.0,2.0,3.0,6.0})
         If (feature 3 <= 8454.5)
          If (feature 7 in {4.0})
           If (feature 5 <= 6.835000038146973)
            Predict: 8064.857142857143
           Else (feature 5 > 6.835000038146973)
            Predict: 7770.111111111111
          Else (feature 7 not in {4.0})
           If (feature 1 <= 59.45000076293945)
            Predict: 8072.0
           Else (feature 1 > 59.45000076293945)
            Predict: 8345.5
         Else (feature 3 > 8454.5)
          If (feature 0 <= 1.2949999570846558)
           If (feature 2 <= 55.04999923706055)
            Predict: 9006.75
           Else (feature 2 > 55.04999923706055)
            Predict: 9182.577777777778
          Else (feature 0 > 1.2949999570846558)
           If (feature 5 <= 6.994999885559082)
            Predict: 9232.0
           Else (feature 5 > 6.994999885559082)
            Predict: 9572.0
    Else (feature 4 > 7.015000104904175)
     If (feature 3 <= 7236.0)
      If (feature 7 in {7.0})
       If (feature 3 <= 5201.5)
        If (feature 8 in {0.0,2.0,4.0,5.0})
         If (feature 1 <= 66.54999923706055)
          If (feature 3 <= 4146.5)
           If (feature 8 in {0.0})
            Predict: 3577.75
           Else (feature 8 not in {0.0})
            Predict: 3766.0
          Else (feature 3 > 4146.5)
           If (feature 2 <= 55.04999923706055)
            Predict: 4240.333333333333
           Else (feature 2 > 55.04999923706055)
            Predict: 4729.733333333334
         Else (feature 1 > 66.54999923706055)
          Predict: 3745.0
        Else (feature 8 not in {0.0,2.0,4.0,5.0})
         If (feature 3 <= 4820.0)
          If (feature 3 <= 4146.5)
           If (feature 0 <= 1.5049999952316284)
            Predict: 4022.0
           Else (feature 0 > 1.5049999952316284)
            Predict: 3765.0
          Else (feature 3 > 4146.5)
           If (feature 4 <= 7.305000066757202)
            Predict: 4492.0
           Else (feature 4 > 7.305000066757202)
            Predict: 4404.0
         Else (feature 3 > 4820.0)
          If (feature 1 <= 62.45000076293945)
           If (feature 0 <= 1.5649999976158142)
            Predict: 5010.875
           Else (feature 0 > 1.5649999976158142)
            Predict: 5046.2
          Else (feature 1 > 62.45000076293945)
           If (feature 6 <= 4.615000009536743)
            Predict: 5141.0
           Else (feature 6 > 4.615000009536743)
            Predict: 5170.0
       Else (feature 3 > 5201.5)
        If (feature 8 in {1.0,4.0,5.0,6.0})
         If (feature 1 <= 66.54999923706055)
          If (feature 9 in {0.0})
           If (feature 8 in {5.0})
            Predict: 5593.0
           Else (feature 8 not in {5.0})
            Predict: 6303.0
          Else (feature 9 not in {0.0})
           If (feature 5 <= 7.484999895095825)
            Predict: 6222.75
           Else (feature 5 > 7.484999895095825)
            Predict: 5914.833333333333
         Else (feature 1 > 66.54999923706055)
          If (feature 2 <= 58.04999923706055)
           If (feature 8 in {5.0})
            Predict: 5696.0
           Else (feature 8 not in {5.0})
            Predict: 5817.0
          Else (feature 2 > 58.04999923706055)
           Predict: 5240.0
        Else (feature 8 not in {1.0,4.0,5.0,6.0})
         If (feature 5 <= 7.484999895095825)
          If (feature 2 <= 61.10000038146973)
           If (feature 1 <= 61.95000076293945)
            Predict: 5976.0
           Else (feature 1 > 61.95000076293945)
            Predict: 6151.333333333333
          Else (feature 2 > 61.10000038146973)
           Predict: 5361.5
         Else (feature 5 > 7.484999895095825)
          If (feature 1 <= 64.04999923706055)
           If (feature 3 <= 6376.5)
            Predict: 6280.0
           Else (feature 3 > 6376.5)
            Predict: 6794.7
          Else (feature 1 > 64.04999923706055)
           If (feature 3 <= 6376.5)
            Predict: 6006.0
           Else (feature 3 > 6376.5)
            Predict: 6771.529411764706
      Else (feature 7 not in {7.0})
       If (feature 3 <= 5716.5)
        If (feature 8 in {2.0,3.0,4.0})
         If (feature 2 <= 60.14999961853027)
          If (feature 3 <= 5201.5)
           If (feature 3 <= 4455.5)
            Predict: 4022.5
           Else (feature 3 > 4455.5)
            Predict: 4604.333333333333
          Else (feature 3 > 5201.5)
           If (feature 0 <= 1.5049999952316284)
            Predict: 5571.0
           Else (feature 0 > 1.5049999952316284)
            Predict: 5395.0
         Else (feature 2 > 60.14999961853027)
          If (feature 2 <= 61.10000038146973)
           If (feature 6 <= 4.224999904632568)
            Predict: 2845.0
           Else (feature 6 > 4.224999904632568)
            Predict: 4732.0
          Else (feature 2 > 61.10000038146973)
           If (feature 6 <= 3.965000033378601)
            Predict: 4903.0
           Else (feature 6 > 3.965000033378601)
            Predict: 4543.0
        Else (feature 8 not in {2.0,3.0,4.0})
         If (feature 3 <= 5201.5)
          If (feature 3 <= 4820.0)
           If (feature 1 <= 60.75)
            Predict: 4500.142857142857
           Else (feature 1 > 60.75)
            Predict: 4679.0
          Else (feature 3 > 4820.0)
           If (feature 2 <= 58.04999923706055)
            Predict: 5092.4
           Else (feature 2 > 58.04999923706055)
            Predict: 5002.8
         Else (feature 3 > 5201.5)
          If (feature 9 in {0.0,1.0,3.0,4.0})
           If (feature 1 <= 59.45000076293945)
            Predict: 5348.6
           Else (feature 1 > 59.45000076293945)
            Predict: 5453.5
          Else (feature 9 not in {0.0,1.0,3.0,4.0})
           If (feature 6 <= 4.224999904632568)
            Predict: 5500.0
           Else (feature 6 > 4.224999904632568)
            Predict: 5580.8
       Else (feature 3 > 5716.5)
        If (feature 6 <= 4.315000057220459)
         If (feature 7 in {1.0,2.0})
          If (feature 1 <= 58.75)
           If (feature 3 <= 6376.5)
            Predict: 5977.266666666666
           Else (feature 3 > 6376.5)
            Predict: 6397.75
          Else (feature 1 > 58.75)
           If (feature 9 in {0.0,1.0,3.0})
            Predict: 6301.717948717948
           Else (feature 9 not in {0.0,1.0,3.0})
            Predict: 6878.0
         Else (feature 7 not in {1.0,2.0})
          If (feature 3 <= 6376.5)
           If (feature 8 in {1.0,6.0})
            Predict: 5825.5
           Else (feature 8 not in {1.0,6.0})
            Predict: 6107.333333333333
          Else (feature 3 > 6376.5)
           If (feature 4 <= 7.305000066757202)
            Predict: 6726.125
           Else (feature 4 > 7.305000066757202)
            Predict: 7197.0
        Else (feature 6 > 4.315000057220459)
         If (feature 8 in {3.0,5.0,6.0})
          If (feature 9 in {0.0,1.0,3.0})
           If (feature 0 <= 1.5649999976158142)
            Predict: 6413.7037037037035
           Else (feature 0 > 1.5649999976158142)
            Predict: 6762.428571428572
          Else (feature 9 not in {0.0,1.0,3.0})
           If (feature 3 <= 6376.5)
            Predict: 6131.666666666667
           Else (feature 3 > 6376.5)
            Predict: 6852.489795918367
         Else (feature 8 not in {3.0,5.0,6.0})
          If (feature 3 <= 6376.5)
           If (feature 4 <= 7.484999895095825)
            Predict: 6116.909090909091
           Else (feature 4 > 7.484999895095825)
            Predict: 6006.0
          Else (feature 3 > 6376.5)
           If (feature 7 in {0.0,2.0})
            Predict: 6851.087719298245
           Else (feature 7 not in {0.0,2.0})
            Predict: 7214.0
     Else (feature 3 > 7236.0)
      If (feature 4 <= 7.484999895095825)
       If (feature 3 <= 8454.5)
        If (feature 7 in {1.0,2.0})
         If (feature 9 in {1.0,2.0})
          If (feature 8 in {2.0,5.0,6.0})
           If (feature 4 <= 7.305000066757202)
            Predict: 7378.7692307692305
           Else (feature 4 > 7.305000066757202)
            Predict: 7587.673913043478
          Else (feature 8 not in {2.0,5.0,6.0})
           If (feature 5 <= 7.305000066757202)
            Predict: 7683.068181818182
           Else (feature 5 > 7.305000066757202)
            Predict: 7971.56
         Else (feature 9 not in {1.0,2.0})
          If (feature 2 <= 59.20000076293945)
           If (feature 6 <= 4.515000104904175)
            Predict: 7641.0
           Else (feature 6 > 4.515000104904175)
            Predict: 7814.181818181818
          Else (feature 2 > 59.20000076293945)
           If (feature 7 in {2.0})
            Predict: 7876.928571428572
           Else (feature 7 not in {2.0})
            Predict: 8171.375
        Else (feature 7 not in {1.0,2.0})
         If (feature 7 in {0.0,4.0})
          If (feature 2 <= 56.04999923706055)
           If (feature 9 in {1.0})
            Predict: 7497.0
           Else (feature 9 not in {1.0})
            Predict: 8067.1875
          Else (feature 2 > 56.04999923706055)
           If (feature 9 in {1.0,2.0,4.0})
            Predict: 7781.044117647059
           Else (feature 9 not in {1.0,2.0,4.0})
            Predict: 7898.652173913043
         Else (feature 7 not in {0.0,4.0})
          If (feature 8 in {5.0})
           If (feature 2 <= 57.04999923706055)
            Predict: 7617.0
           Else (feature 2 > 57.04999923706055)
            Predict: 7592.0
          Else (feature 8 not in {5.0})
           If (feature 2 <= 60.14999961853027)
            Predict: 8051.347826086957
           Else (feature 2 > 60.14999961853027)
            Predict: 8320.625
       Else (feature 3 > 8454.5)
        If (feature 1 <= 62.64999961853027)
         If (feature 8 in {6.0})
          If (feature 5 <= 7.484999895095825)
           If (feature 7 in {1.0,3.0,4.0})
            Predict: 8677.346153846154
           Else (feature 7 not in {1.0,3.0,4.0})
            Predict: 9107.2
          Else (feature 5 > 7.484999895095825)
           If (feature 7 in {1.0})
            Predict: 9681.0
           Else (feature 7 not in {1.0})
            Predict: 9683.0
         Else (feature 8 not in {6.0})
          If (feature 5 <= 7.484999895095825)
           If (feature 6 <= 4.144999980926514)
            Predict: 9541.0
           Else (feature 6 > 4.144999980926514)
            Predict: 8990.792
          Else (feature 5 > 7.484999895095825)
           If (feature 6 <= 2.6149998903274536)
            Predict: 8971.0
           Else (feature 6 > 2.6149998903274536)
            Predict: 9395.2
        Else (feature 1 > 62.64999961853027)
         If (feature 8 in {6.0})
          If (feature 1 <= 62.75)
           Predict: 8479.0
          Else (feature 1 > 62.75)
           If (feature 7 in {1.0,3.0})
            Predict: 8654.5
           Else (feature 7 not in {1.0,3.0})
            Predict: 9382.0
         Else (feature 8 not in {6.0})
          If (feature 4 <= 7.305000066757202)
           If (feature 7 in {3.0})
            Predict: 8822.0
           Else (feature 7 not in {3.0})
            Predict: 9108.533333333333
          Else (feature 4 > 7.305000066757202)
           If (feature 7 in {2.0})
            Predict: 9139.90909090909
           Else (feature 7 not in {2.0})
            Predict: 9415.666666666666
      Else (feature 4 > 7.484999895095825)
       If (feature 3 <= 8454.5)
        If (feature 4 <= 7.984999895095825)
         If (feature 7 in {3.0,7.0})
          If (feature 5 <= 5.775000095367432)
           Predict: 8133.0
          Else (feature 5 > 5.775000095367432)
           If (feature 8 in {3.0})
            Predict: 7391.25
           Else (feature 8 not in {3.0})
            Predict: 7802.0
         Else (feature 7 not in {3.0,7.0})
          If (feature 8 in {5.0})
           If (feature 6 <= 4.615000009536743)
            Predict: 7854.666666666667
           Else (feature 6 > 4.615000009536743)
            Predict: 7569.142857142857
          Else (feature 8 not in {5.0})
           If (feature 7 in {0.0,1.0,2.0})
            Predict: 7969.983333333334
           Else (feature 7 not in {0.0,1.0,2.0})
            Predict: 8214.5
        Else (feature 4 > 7.984999895095825)
         If (feature 8 in {2.0,3.0})
          If (feature 8 in {2.0})
           If (feature 0 <= 2.0049999952316284)
            Predict: 7258.0
           Else (feature 0 > 2.0049999952316284)
            Predict: 7294.0
          Else (feature 8 not in {2.0})
           Predict: 7421.0
         Else (feature 8 not in {2.0,3.0})
          Predict: 7934.0
       Else (feature 3 > 8454.5)
        If (feature 8 in {0.0,4.0,6.0})
         If (feature 5 <= 7.974999904632568)
          If (feature 0 <= 1.5649999976158142)
           If (feature 7 in {3.0})
            Predict: 8548.0
           Else (feature 7 not in {3.0})
            Predict: 8807.285714285714
          Else (feature 0 > 1.5649999976158142)
           If (feature 2 <= 56.04999923706055)
            Predict: 8934.75
           Else (feature 2 > 56.04999923706055)
            Predict: 9006.96
         Else (feature 5 > 7.974999904632568)
          If (feature 0 <= 2.0049999952316284)
           Predict: 9296.0
          Else (feature 0 > 2.0049999952316284)
           Predict: 9482.0
        Else (feature 8 not in {0.0,4.0,6.0})
         If (feature 9 in {0.0,1.0,3.0,4.0})
          If (feature 5 <= 7.484999895095825)
           If (feature 1 <= 62.35000038146973)
            Predict: 9210.05
           Else (feature 1 > 62.35000038146973)
            Predict: 9548.0
          Else (feature 5 > 7.484999895095825)
           If (feature 9 in {0.0})
            Predict: 8925.3125
           Else (feature 9 not in {0.0})
            Predict: 9075.234567901234
         Else (feature 9 not in {0.0,1.0,3.0,4.0})
          If (feature 8 in {2.0,5.0})
           If (feature 2 <= 57.04999923706055)
            Predict: 9011.5
           Else (feature 2 > 57.04999923706055)
            Predict: 9270.727272727272
          Else (feature 8 not in {2.0,5.0})
           If (feature 1 <= 58.75)
            Predict: 9077.0
           Else (feature 1 > 58.75)
            Predict: 9502.57142857143
   Else (feature 3 > 9692.5)
    If (feature 5 <= 7.974999904632568)
     If (feature 3 <= 14212.0)
      If (feature 4 <= 7.015000104904175)
       If (feature 9 in {1.0,2.0,3.0})
        If (feature 8 in {0.0,4.0})
         If (feature 3 <= 11430.5)
          If (feature 1 <= 62.45000076293945)
           If (feature 7 in {1.0,3.0,4.0,6.0})
            Predict: 10258.848484848484
           Else (feature 7 not in {1.0,3.0,4.0,6.0})
            Predict: 11029.75
          Else (feature 1 > 62.45000076293945)
           If (feature 6 <= 4.005000114440918)
            Predict: 10453.0
           Else (feature 6 > 4.005000114440918)
            Predict: 9735.714285714286
         Else (feature 3 > 11430.5)
          If (feature 5 <= 6.454999923706055)
           Predict: 13312.0
          Else (feature 5 > 6.454999923706055)
           If (feature 1 <= 59.95000076293945)
            Predict: 12152.0
           Else (feature 1 > 59.95000076293945)
            Predict: 11717.0
        Else (feature 8 not in {0.0,4.0})
         If (feature 0 <= 1.1449999809265137)
          If (feature 7 in {3.0,4.0})
           If (feature 4 <= 6.724999904632568)
            Predict: 9923.07142857143
           Else (feature 4 > 6.724999904632568)
            Predict: 11001.0
          Else (feature 7 not in {3.0,4.0})
           If (feature 6 <= 4.075000047683716)
            Predict: 10703.894736842105
           Else (feature 6 > 4.075000047683716)
            Predict: 11427.8
         Else (feature 0 > 1.1449999809265137)
          If (feature 3 <= 11430.5)
           If (feature 7 in {3.0})
            Predict: 10368.391304347826
           Else (feature 7 not in {3.0})
            Predict: 10921.785714285714
          Else (feature 3 > 11430.5)
           If (feature 7 in {4.0})
            Predict: 12056.42857142857
           Else (feature 7 not in {4.0})
            Predict: 13091.833333333334
       Else (feature 9 not in {1.0,2.0,3.0})
        If (feature 6 <= 4.315000057220459)
         If (feature 3 <= 11430.5)
          If (feature 8 in {0.0,2.0,4.0})
           If (feature 6 <= 4.0350000858306885)
            Predict: 10758.095238095239
           Else (feature 6 > 4.0350000858306885)
            Predict: 10379.125
          Else (feature 8 not in {0.0,2.0,4.0})
           If (feature 5 <= 6.7149999141693115)
            Predict: 10624.173913043478
           Else (feature 5 > 6.7149999141693115)
            Predict: 11117.92857142857
         Else (feature 3 > 11430.5)
          If (feature 1 <= 61.85000038146973)
           If (feature 7 in {3.0,6.0})
            Predict: 11979.111111111111
           Else (feature 7 not in {3.0,6.0})
            Predict: 12944.083333333334
          Else (feature 1 > 61.85000038146973)
           If (feature 0 <= 1.0850000381469727)
            Predict: 11770.777777777777
           Else (feature 0 > 1.0850000381469727)
            Predict: 12523.318181818182
        Else (feature 6 > 4.315000057220459)
         If (feature 3 <= 11430.5)
          If (feature 1 <= 62.25)
           If (feature 1 <= 62.04999923706055)
            Predict: 10359.0
           Else (feature 1 > 62.04999923706055)
            Predict: 10090.0
          Else (feature 1 > 62.25)
           If (feature 0 <= 1.2949999570846558)
            Predict: 10865.0
           Else (feature 0 > 1.2949999570846558)
            Predict: 11419.0
         Else (feature 3 > 11430.5)
          If (feature 0 <= 1.2949999570846558)
           If (feature 7 in {3.0})
            Predict: 13485.0
           Else (feature 7 not in {3.0})
            Predict: 13866.0
          Else (feature 0 > 1.2949999570846558)
           If (feature 8 in {0.0})
            Predict: 11633.0
           Else (feature 8 not in {0.0})
            Predict: 12629.0
      Else (feature 4 > 7.015000104904175)
       If (feature 6 <= 4.315000057220459)
        If (feature 8 in {3.0,4.0})
         If (feature 8 in {3.0})
          Predict: 9735.0
         Else (feature 8 not in {3.0})
          If (feature 0 <= 1.2949999570846558)
           Predict: 9880.4
          Else (feature 0 > 1.2949999570846558)
           Predict: 9874.0
        Else (feature 8 not in {3.0,4.0})
         If (feature 9 in {1.0})
          If (feature 8 in {0.0,2.0})
           If (feature 2 <= 59.20000076293945)
            Predict: 10158.23076923077
           Else (feature 2 > 59.20000076293945)
            Predict: 10824.75
          Else (feature 8 not in {0.0,2.0})
           Predict: 12100.0
         Else (feature 9 not in {1.0})
          If (feature 3 <= 11430.5)
           If (feature 1 <= 60.54999923706055)
            Predict: 10219.5
           Else (feature 1 > 60.54999923706055)
            Predict: 11098.166666666666
          Else (feature 3 > 11430.5)
           If (feature 0 <= 1.2949999570846558)
            Predict: 14095.0
           Else (feature 0 > 1.2949999570846558)
            Predict: 11549.8
       Else (feature 6 > 4.315000057220459)
        If (feature 8 in {3.0,5.0,6.0})
         If (feature 3 <= 11430.5)
          If (feature 9 in {3.0})
           If (feature 1 <= 58.75)
            Predict: 9896.416666666666
           Else (feature 1 > 58.75)
            Predict: 10301.0
          Else (feature 9 not in {3.0})
           If (feature 7 in {0.0,2.0})
            Predict: 10439.364963503649
           Else (feature 7 not in {0.0,2.0})
            Predict: 10679.547244094489
         Else (feature 3 > 11430.5)
          If (feature 5 <= 7.484999895095825)
           If (feature 7 in {0.0,1.0,3.0})
            Predict: 11712.80487804878
           Else (feature 7 not in {0.0,1.0,3.0})
            Predict: 12357.944444444445
          Else (feature 5 > 7.484999895095825)
           If (feature 6 <= 4.615000009536743)
            Predict: 12081.833333333334
           Else (feature 6 > 4.615000009536743)
            Predict: 12657.678362573099
        Else (feature 8 not in {3.0,5.0,6.0})
         If (feature 4 <= 7.305000066757202)
          If (feature 0 <= 1.5049999952316284)
           If (feature 1 <= 63.04999923706055)
            Predict: 11818.2
           Else (feature 1 > 63.04999923706055)
            Predict: 11132.941176470587
          Else (feature 0 > 1.5049999952316284)
           If (feature 8 in {1.0,4.0})
            Predict: 11417.387096774193
           Else (feature 8 not in {1.0,4.0})
            Predict: 12461.04918032787
         Else (feature 4 > 7.305000066757202)
          If (feature 7 in {0.0,2.0})
           If (feature 7 in {2.0})
            Predict: 11077.517006802722
           Else (feature 7 not in {2.0})
            Predict: 11954.080536912752
          Else (feature 7 not in {0.0,2.0})
           If (feature 3 <= 11430.5)
            Predict: 11003.235294117647
           Else (feature 3 > 11430.5)
            Predict: 13242.854748603351
     Else (feature 3 > 14212.0)
      If (feature 3 <= 17965.0)
       If (feature 6 <= 4.615000009536743)
        If (feature 8 in {0.0,1.0,2.0})
         If (feature 5 <= 6.994999885559082)
          If (feature 2 <= 57.04999923706055)
           If (feature 1 <= 61.64999961853027)
            Predict: 14505.375
           Else (feature 1 > 61.64999961853027)
            Predict: 15497.0
          Else (feature 2 > 57.04999923706055)
           Predict: 15185.0
         Else (feature 5 > 6.994999885559082)
          If (feature 7 in {0.0,1.0})
           If (feature 2 <= 53.95000076293945)
            Predict: 15649.0
           Else (feature 2 > 53.95000076293945)
            Predict: 14674.898550724638
          Else (feature 7 not in {0.0,1.0})
           If (feature 8 in {0.0})
            Predict: 15377.744444444445
           Else (feature 8 not in {0.0})
            Predict: 15906.363636363636
        Else (feature 8 not in {0.0,1.0,2.0})
         If (feature 7 in {1.0,4.0,6.0})
          If (feature 4 <= 7.484999895095825)
           If (feature 4 <= 6.8450000286102295)
            Predict: 15696.75
           Else (feature 4 > 6.8450000286102295)
            Predict: 15056.92857142857
          Else (feature 4 > 7.484999895095825)
           Predict: 16787.0
         Else (feature 7 not in {1.0,4.0,6.0})
          If (feature 0 <= 1.215000033378601)
           Predict: 15767.0
          Else (feature 0 > 1.215000033378601)
           If (feature 9 in {0.0})
            Predict: 16627.333333333332
           Else (feature 9 not in {0.0})
            Predict: 17728.5
       Else (feature 6 > 4.615000009536743)
        If (feature 5 <= 7.484999895095825)
         If (feature 1 <= 62.45000076293945)
          If (feature 2 <= 55.04999923706055)
           Predict: 14495.0
          Else (feature 2 > 55.04999923706055)
           If (feature 7 in {3.0})
            Predict: 14451.0
           Else (feature 7 not in {3.0})
            Predict: 14948.125
         Else (feature 1 > 62.45000076293945)
          If (feature 0 <= 1.5649999976158142)
           If (feature 1 <= 64.04999923706055)
            Predict: 16012.0
           Else (feature 1 > 64.04999923706055)
            Predict: 15476.0
          Else (feature 0 > 1.5649999976158142)
           If (feature 9 in {2.0})
            Predict: 14400.0
           Else (feature 9 not in {2.0})
            Predict: 15721.0
        Else (feature 5 > 7.484999895095825)
         If (feature 1 <= 62.95000076293945)
          If (feature 8 in {3.0,4.0})
           If (feature 1 <= 62.85000038146973)
            Predict: 15174.939393939394
           Else (feature 1 > 62.85000038146973)
            Predict: 17838.0
          Else (feature 8 not in {3.0,4.0})
           If (feature 1 <= 60.85000038146973)
            Predict: 15467.6
           Else (feature 1 > 60.85000038146973)
            Predict: 16462.70909090909
         Else (feature 1 > 62.95000076293945)
          If (feature 6 <= 4.934999942779541)
           If (feature 9 in {2.0})
            Predict: 14557.5
           Else (feature 9 not in {2.0})
            Predict: 16157.5
          Else (feature 6 > 4.934999942779541)
           If (feature 9 in {1.0,4.0})
            Predict: 15386.0
           Else (feature 9 not in {1.0,4.0})
            Predict: 15989.78947368421
      Else (feature 3 > 17965.0)
       If (feature 7 in {1.0,3.0,4.0})
        If (feature 1 <= 62.14999961853027)
         If (feature 2 <= 57.04999923706055)
          If (feature 1 <= 61.95000076293945)
           If (feature 0 <= 1.5649999976158142)
            Predict: 18729.0
           Else (feature 0 > 1.5649999976158142)
            Predict: 18107.0
          Else (feature 1 > 61.95000076293945)
           Predict: 18711.666666666668
         Else (feature 2 > 57.04999923706055)
          If (feature 8 in {1.0})
           Predict: 18342.0
          Else (feature 8 not in {1.0})
           If (feature 1 <= 58.75)
            Predict: 18398.0
           Else (feature 1 > 58.75)
            Predict: 18419.0
        Else (feature 1 > 62.14999961853027)
         If (feature 6 <= 4.934999942779541)
          If (feature 9 in {0.0,2.0})
           If (feature 1 <= 62.35000038146973)
            Predict: 18055.0
           Else (feature 1 > 62.35000038146973)
            Predict: 18147.0
          Else (feature 9 not in {0.0,2.0})
           Predict: 18377.0
         Else (feature 6 > 4.934999942779541)
          If (feature 9 in {0.0,1.0})
           If (feature 9 in {0.0})
            Predict: 18172.0
           Else (feature 9 not in {0.0})
            Predict: 18206.0
          Else (feature 9 not in {0.0,1.0})
           If (feature 8 in {5.0})
            Predict: 18500.0
           Else (feature 8 not in {5.0})
            Predict: 18515.0
       Else (feature 7 not in {1.0,3.0,4.0})
        If (feature 0 <= 1.1449999809265137)
         If (feature 0 <= 1.0850000381469727)
          Predict: 18279.0
         Else (feature 0 > 1.0850000381469727)
          Predict: 18112.0
        Else (feature 0 > 1.1449999809265137)
         If (feature 7 in {5.0})
          If (feature 9 in {0.0})
           If (feature 8 in {0.0})
            Predict: 18369.5
           Else (feature 8 not in {0.0})
            Predict: 18682.0
          Else (feature 9 not in {0.0})
           Predict: 18777.0
         Else (feature 7 not in {5.0})
          If (feature 5 <= 7.484999895095825)
           If (feature 2 <= 57.04999923706055)
            Predict: 18753.0
           Else (feature 2 > 57.04999923706055)
            Predict: 18563.0
          Else (feature 5 > 7.484999895095825)
           If (feature 8 in {3.0})
            Predict: 18781.0
           Else (feature 8 not in {3.0})
            Predict: 18818.0
    Else (feature 5 > 7.974999904632568)
     If (feature 7 in {2.0,7.0})
      If (feature 8 in {3.0,5.0,6.0})
       If (feature 8 in {6.0})
        If (feature 3 <= 14212.0)
         If (feature 3 <= 11430.5)
          If (feature 6 <= 5.275000095367432)
           If (feature 5 <= 8.515000343322754)
            Predict: 11130.833333333334
           Else (feature 5 > 8.515000343322754)
            Predict: 10716.0
          Else (feature 6 > 5.275000095367432)
           If (feature 9 in {2.0})
            Predict: 10437.0
           Else (feature 9 not in {2.0})
            Predict: 10628.0
         Else (feature 3 > 11430.5)
          If (feature 4 <= 7.984999895095825)
           If (feature 1 <= 62.64999961853027)
            Predict: 12342.0
           Else (feature 1 > 62.64999961853027)
            Predict: 11534.25
          Else (feature 4 > 7.984999895095825)
           If (feature 2 <= 55.04999923706055)
            Predict: 11876.8
           Else (feature 2 > 55.04999923706055)
            Predict: 12747.607843137255
        Else (feature 3 > 14212.0)
         If (feature 0 <= 2.3249999284744263)
          If (feature 1 <= 62.54999923706055)
           Predict: 14666.0
          Else (feature 1 > 62.54999923706055)
           Predict: 14500.0
         Else (feature 0 > 2.3249999284744263)
          If (feature 3 <= 17965.0)
           If (feature 4 <= 8.539999961853027)
            Predict: 14652.0
           Else (feature 4 > 8.539999961853027)
            Predict: 16318.357142857143
          Else (feature 3 > 17965.0)
           If (feature 7 in {7.0})
            Predict: 18274.5
           Else (feature 7 not in {7.0})
            Predict: 18710.0
       Else (feature 8 not in {6.0})
        If (feature 1 <= 60.25)
         If (feature 0 <= 2.3249999284744263)
          If (feature 3 <= 14212.0)
           If (feature 2 <= 60.14999961853027)
            Predict: 12287.466666666667
           Else (feature 2 > 60.14999961853027)
            Predict: 13405.333333333334
          Else (feature 3 > 14212.0)
           If (feature 1 <= 59.95000076293945)
            Predict: 15265.9375
           Else (feature 1 > 59.95000076293945)
            Predict: 16547.5
         Else (feature 0 > 2.3249999284744263)
          If (feature 7 in {7.0})
           Predict: 10863.0
          Else (feature 7 not in {7.0})
           If (feature 9 in {1.0,4.0})
            Predict: 15825.666666666666
           Else (feature 9 not in {1.0,4.0})
            Predict: 17923.0
        Else (feature 1 > 60.25)
         If (feature 6 <= 5.275000095367432)
          If (feature 2 <= 57.04999923706055)
           If (feature 1 <= 61.04999923706055)
            Predict: 15319.6
           Else (feature 1 > 61.04999923706055)
            Predict: 14031.295774647888
          Else (feature 2 > 57.04999923706055)
           If (feature 4 <= 8.539999961853027)
            Predict: 14732.504273504273
           Else (feature 4 > 8.539999961853027)
            Predict: 17514.333333333332
         Else (feature 6 > 5.275000095367432)
          If (feature 3 <= 14212.0)
           If (feature 3 <= 11430.5)
            Predict: 10424.0
           Else (feature 3 > 11430.5)
            Predict: 12975.21052631579
          Else (feature 3 > 14212.0)
           If (feature 2 <= 57.04999923706055)
            Predict: 17098.7
           Else (feature 2 > 57.04999923706055)
            Predict: 16178.961538461539
      Else (feature 8 not in {3.0,5.0,6.0})
       If (feature 2 <= 59.20000076293945)
        If (feature 3 <= 14212.0)
         If (feature 7 in {7.0})
          If (feature 0 <= 2.0049999952316284)
           Predict: 10494.0
          Else (feature 0 > 2.0049999952316284)
           If (feature 1 <= 61.04999923706055)
            Predict: 10453.0
           Else (feature 1 > 61.04999923706055)
            Predict: 10038.25
         Else (feature 7 not in {7.0})
          If (feature 2 <= 57.04999923706055)
           If (feature 9 in {1.0,2.0,4.0})
            Predict: 12214.555555555555
           Else (feature 9 not in {1.0,2.0,4.0})
            Predict: 12793.666666666666
          Else (feature 2 > 57.04999923706055)
           If (feature 0 <= 2.0049999952316284)
            Predict: 12358.5
           Else (feature 0 > 2.0049999952316284)
            Predict: 13276.851851851852
        Else (feature 3 > 14212.0)
         If (feature 3 <= 17965.0)
          If (feature 4 <= 8.539999961853027)
           If (feature 2 <= 58.04999923706055)
            Predict: 16045.240963855422
           Else (feature 2 > 58.04999923706055)
            Predict: 16834.125
          Else (feature 4 > 8.539999961853027)
           If (feature 5 <= 8.515000343322754)
            Predict: 14648.0
           Else (feature 5 > 8.515000343322754)
            Predict: 17146.4
         Else (feature 3 > 17965.0)
          If (feature 6 <= 4.934999942779541)
           If (feature 9 in {2.0})
            Predict: 18115.0
           Else (feature 9 not in {2.0})
            Predict: 18120.0
          Else (feature 6 > 4.934999942779541)
           If (feature 0 <= 2.3249999284744263)
            Predict: 18382.976744186046
           Else (feature 0 > 2.3249999284744263)
            Predict: 18688.0
       Else (feature 2 > 59.20000076293945)
        If (feature 3 <= 14212.0)
         If (feature 9 in {2.0})
          If (feature 7 in {7.0})
           If (feature 6 <= 4.934999942779541)
            Predict: 9789.0
           Else (feature 6 > 4.934999942779541)
            Predict: 10340.0
          Else (feature 7 not in {7.0})
           If (feature 8 in {2.0})
            Predict: 12099.0
           Else (feature 8 not in {2.0})
            Predict: 12639.0
         Else (feature 9 not in {2.0})
          If (feature 3 <= 11430.5)
           Predict: 10833.0
          Else (feature 3 > 11430.5)
           If (feature 2 <= 61.10000038146973)
            Predict: 13211.588235294117
           Else (feature 2 > 61.10000038146973)
            Predict: 11946.0
        Else (feature 3 > 14212.0)
         If (feature 9 in {3.0,4.0})
          If (feature 2 <= 60.14999961853027)
           Predict: 14745.0
          Else (feature 2 > 60.14999961853027)
           If (feature 1 <= 58.75)
            Predict: 16995.5
           Else (feature 1 > 58.75)
            Predict: 15954.0
         Else (feature 9 not in {3.0,4.0})
          If (feature 3 <= 17965.0)
           If (feature 8 in {0.0,1.0,4.0})
            Predict: 16455.625
           Else (feature 8 not in {0.0,1.0,4.0})
            Predict: 17188.0
          Else (feature 3 > 17965.0)
           If (feature 1 <= 61.04999923706055)
            Predict: 18124.0
           Else (feature 1 > 61.04999923706055)
            Predict: 18042.25
     Else (feature 7 not in {2.0,7.0})
      If (feature 3 <= 14212.0)
       If (feature 7 in {0.0,3.0})
        If (feature 8 in {5.0,6.0})
         If (feature 2 <= 60.14999961853027)
          If (feature 0 <= 2.3249999284744263)
           If (feature 9 in {0.0,2.0})
            Predict: 12578.0
           Else (feature 9 not in {0.0,2.0})
            Predict: 13235.782608695652
          Else (feature 0 > 2.3249999284744263)
           If (feature 4 <= 8.539999961853027)
            Predict: 12416.0
           Else (feature 4 > 8.539999961853027)
            Predict: 11638.0
         Else (feature 2 > 60.14999961853027)
          If (feature 9 in {4.0})
           Predict: 11322.0
          Else (feature 9 not in {4.0})
           If (feature 9 in {2.0})
            Predict: 12257.0
           Else (feature 9 not in {2.0})
            Predict: 12329.0
        Else (feature 8 not in {5.0,6.0})
         If (feature 2 <= 60.14999961853027)
          If (feature 1 <= 59.95000076293945)
           If (feature 2 <= 58.04999923706055)
            Predict: 13905.0
           Else (feature 2 > 58.04999923706055)
            Predict: 13986.0
          Else (feature 1 > 59.95000076293945)
           If (feature 0 <= 2.0049999952316284)
            Predict: 13675.0
           Else (feature 0 > 2.0049999952316284)
            Predict: 13754.166666666666
         Else (feature 2 > 60.14999961853027)
          If (feature 0 <= 2.0049999952316284)
           Predict: 12923.0
          Else (feature 0 > 2.0049999952316284)
           Predict: 12841.5
       Else (feature 7 not in {0.0,3.0})
        If (feature 3 <= 11430.5)
         Predict: 11059.0
        Else (feature 3 > 11430.5)
         If (feature 1 <= 61.14999961853027)
          If (feature 1 <= 58.75)
           Predict: 14042.0
          Else (feature 1 > 58.75)
           If (feature 6 <= 4.934999942779541)
            Predict: 13157.555555555555
           Else (feature 6 > 4.934999942779541)
            Predict: 13191.285714285714
         Else (feature 1 > 61.14999961853027)
          If (feature 6 <= 4.934999942779541)
           Predict: 12683.0
          Else (feature 6 > 4.934999942779541)
           If (feature 1 <= 61.54999923706055)
            Predict: 13535.25
           Else (feature 1 > 61.54999923706055)
            Predict: 13837.416666666666
      Else (feature 3 > 14212.0)
       If (feature 4 <= 8.539999961853027)
        If (feature 3 <= 17965.0)
         If (feature 1 <= 62.95000076293945)
          If (feature 7 in {1.0,3.0})
           If (feature 2 <= 57.04999923706055)
            Predict: 16542.60714285714
           Else (feature 2 > 57.04999923706055)
            Predict: 16230.301587301587
          Else (feature 7 not in {1.0,3.0})
           If (feature 5 <= 8.515000343322754)
            Predict: 16703.980392156864
           Else (feature 5 > 8.515000343322754)
            Predict: 17772.0
         Else (feature 1 > 62.95000076293945)
          If (feature 8 in {3.0,5.0,6.0})
           If (feature 7 in {0.0,3.0,4.0})
            Predict: 15163.75
           Else (feature 7 not in {0.0,3.0,4.0})
            Predict: 16488.5
          Else (feature 8 not in {3.0,5.0,6.0})
           Predict: 17724.0
        Else (feature 3 > 17965.0)
         If (feature 7 in {3.0,6.0})
          If (feature 2 <= 59.20000076293945)
           If (feature 1 <= 61.04999923706055)
            Predict: 17986.0
           Else (feature 1 > 61.04999923706055)
            Predict: 18128.0
          Else (feature 2 > 59.20000076293945)
           If (feature 8 in {3.0,5.0})
            Predict: 18180.25
           Else (feature 8 not in {3.0,5.0})
            Predict: 18650.0
         Else (feature 7 not in {3.0,6.0})
          If (feature 1 <= 62.75)
           If (feature 6 <= 5.275000095367432)
            Predict: 18305.78947368421
           Else (feature 6 > 5.275000095367432)
            Predict: 18439.25
          Else (feature 1 > 62.75)
           If (feature 0 <= 2.0049999952316284)
            Predict: 18480.833333333332
           Else (feature 0 > 2.0049999952316284)
            Predict: 18628.85714285714
       Else (feature 4 > 8.539999961853027)
        If (feature 3 <= 17965.0)
         If (feature 2 <= 55.04999923706055)
          Predict: 17955.0
         Else (feature 2 > 55.04999923706055)
          If (feature 8 in {5.0,6.0})
           If (feature 2 <= 56.04999923706055)
            Predict: 15842.5
           Else (feature 2 > 56.04999923706055)
            Predict: 16961.954545454544
          Else (feature 8 not in {5.0,6.0})
           Predict: 17893.0
        Else (feature 3 > 17965.0)
         If (feature 2 <= 56.04999923706055)
          If (feature 9 in {0.0})
           Predict: 18113.0
          Else (feature 9 not in {0.0})
           Predict: 18119.0
         Else (feature 2 > 56.04999923706055)
          If (feature 1 <= 61.25)
           If (feature 1 <= 60.54999923706055)
            Predict: 18237.333333333332
           Else (feature 1 > 60.54999923706055)
            Predict: 18026.0
          Else (feature 1 > 61.25)
           If (feature 2 <= 57.04999923706055)
            Predict: 18692.0
           Else (feature 2 > 57.04999923706055)
            Predict: 18274.285714285714


```


```

```

Your explanation here: The feature carat has value 0.995 and it is being split. The value of carat determines the split. It is a right split 

# Question 6 (5 pts)
Describe if the random forest model MSE score was better or worse than the MSE score from you best model in homework 3.  Include both scores in your description.

Your improvement explanation here:  The train model for linear regression has MSE= 2033430.069693428, and the test model for linear regression has MSE= 2183518.1421358134. Similarly, the train model for random forest regression has MSE= 59599.2120630911  and the test model for random forest regression has MSE=73448.77466708544 . Thus Random Forest Regression seems to be a better regression model due to lower train and test model MSE. 

##### Grading Feedback Cell

# Question 7 (5 pts)
Set the `enable_grid_search` Boolean variable to False in the grading cell at the top of this notebook.  Perform a __Runtime -> factory reset__, __Runtime -> Run all__ test to verify there are no runtime errors.  Leave the `enable_grid_search` variable set to False and turn in your assignment.  This is the kind of thing you should be doing before you turn in every assignment. Remember this for future classes and when you get a job in industry.  This question will be graded as all or nothing.  You ether set the Boolean correct or not.  Additional points will be deducted elsewhere for runtime errors.

In [13]:
if enable_grid_search: 
  from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
  from pyspark.ml.feature import VectorAssembler
  from pyspark.ml import feature
  from pyspark.ml.evaluation import RegressionEvaluator

  from pyspark.ml import Pipeline
  from pyspark.ml.regression import RandomForestRegressor
  from pyspark.ml.evaluation import BinaryClassificationEvaluator
  training_df, testing_df = diamonds_df_xformed.randomSplit([0.7, 0.3],seed=30)
  vc = feature.VectorAssembler(inputCols=['carat','depth','table','price','x','y','z','clarity','color','cut'], outputCol='features')
  rf = RandomForestRegressor(featuresCol='features',labelCol='price')
  rf_pipeline = Pipeline(stages=[vc, rf])
  evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")

  if enable_grid_search:
    paramGrid = (ParamGridBuilder().addGrid(rf.numTrees,[5,20,40]).addGrid(rf.maxDepth,[2,5,10]).build())
    crossval= CrossValidator(estimator= rf_pipeline,estimatorParamMaps=paramGrid,evaluator=evaluator,numFolds=3)
    output = crossval.fit(training_df)
  #predictTrain= model.transform(testing_df)
  #print('MSE:', evaluator.evaluate(predictTrain))
  #predictTrain.show(10)
    print('mse',output.avgMetrics)

pass


# Extra Credit (10 pts)
This homework was intended to take less time to complete and be about half the effort of previous assignments.  This doesn't allow us to explore GBT or deep learning.  

For extra credit, train a GBT or Deep Learning model using a grid search.  Protect the grid search inside the if enable_grid_search statement in the first code cell below.  You are free to use K-Fold cross validation if you wish.  The spark documentation for GBM can be found [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier).  The spark documentation for deep learning can be found [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier)

In the second code cell below, hard code the best model parameters as determined by the grid search in a new pipeline named `best_pipe_2`.  Train and test `best_pipe_2` and save your resulting test MSE in a variable.  Do not use K-Fold cross validation when training best_pipe_2.  

In the third code cell below, create a pandas data frame named `compare_1_df` which contains 2 columns: Model and MSE.  Populate the Model column with model names: LR, RF, GBT or DL.  Populate the score column with the linear regression, random forest, and gradient boosted tree or deep learning test MSE scores. The linear regression score is from homework 3. The random forest score is from the random forest model above.  The GBT or Deep Learning score is from this extra credit problem.  Sort compare_1_df such that the best score is in the first row of the data frame. 

To get full credit, your GBT or deep learning solution should produce a score as good or better than the random forest score above.  In addition, the same rules as above apply where all of your grid search code shall be protected by the enable_grid_search Boolean.  Code that produces a runtime error when enable_grid_search is set to False will get 0 credit.

In [14]:
# Your GBT / Deep Learning grid search code here
from pyspark.ml.regression import GBTRegressor

train, test = diamonds_df_xformed.randomSplit([0.7, 0.3],seed=30)
vac = feature.VectorAssembler(inputCols=['carat','depth','table','price','x','y','z','clarity','color','cut'], outputCol='vfeatures')

gbt = GBTRegressor(featuresCol="vfeatures",labelCol='price')
gbt_pipeline = Pipeline(stages=[vac, gbt])
evaluator2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")

if enable_grid_search:
  gbparamGrid = (ParamGridBuilder().addGrid(gb.maxDepth, [2, 5, 10]).addGrid(gb.maxBins, [10, 20, 40]).addGrid(gb.maxIter, [5, 10, 20]).build())
  #crossval2= CrossValidator(estimator= gbt_pipeline,estimatorParamMaps=gbparamGrid,evaluator=evaluator2,numFolds=3)
  gbcv = CrossValidator(estimator = gb,
                      estimatorParamMaps = gbparamGrid,
                      evaluator = evaluator2,
                      numFolds = 3)

  output2 = gbcv.fit(train)
  print('mse',output2.avgMetrics)


pass

I tried running the code but it gave me Java Server errors. But i am confident that this is how it is suppose to be coded. 

In [ ]:
# your hard coded parameter best model code here


In [ ]:
# Create compare_1_df

In [ ]:
# Grading cell do not modify
display(compare_1_df)